In [1]:
import pandas as pd
import numpy as np
import gc
import time
from datetime import datetime, timedelta
from typing import Dict, List, Tuple, Optional
import warnings
warnings.filterwarnings('ignore')

import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.cluster import MiniBatchKMeans
from sklearn.preprocessing import StandardScaler
from scipy.spatial.distance import cdist
from sklearn.preprocessing import LabelEncoder

In [2]:
train_df = pd.read_parquet('/kaggle/input/amex-data/final_train.parquet')
print(train_df.shape)
train_df['id4'] = pd.to_datetime(train_df['id4'])

train_df
gc.collect()
train_df['id1']

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/amex-data/final_train.parquet'

In [ ]:
df_trans = pd.read_parquet('/kaggle/input/amex-data/add_trans.parquet')
df_event = pd.read_parquet('/kaggle/input/amex-data/add_event.parquet')
df_offer = pd.read_parquet('/kaggle/input/amex-data/offer_metadata.parquet')
gc.collect()

In [ ]:
len(train_df['id2'].unique())

In [ ]:
test_df = pd.read_parquet('/kaggle/input/amex-data/final_test.parquet')
print(test_df.shape)
test_df.head()
gc.collect()

In [ ]:
set(df_offer) & set(df_event)


In [ ]:
# def handle_missing_values(df: pd.DataFrame, missing_threshold: float = 0.95) -> pd.DataFrame:
#     """
#     Handle missing values in a DataFrame
    
#     Parameters:
#     - df: Input DataFrame
#     - missing_threshold: Drop columns with missing ratio >= this threshold (default: 0.95)
    
#     Returns:
#     - DataFrame with missing values handled
#     """
#     df_copy = df.copy()
    
#     # Get all columns (excluding non-feature columns if needed)
#     feature_cols = df_copy.columns.tolist()
    
#     # Track columns to drop
#     cols_to_drop = []
    
#     # Check each column for missing values
#     for col in feature_cols:
#         missing_count = df_copy[col].isna().sum()
#         missing_ratio = missing_count / len(df_copy)
        
#         # Drop columns with too many missing values
#         if missing_ratio >= missing_threshold:
#             cols_to_drop.append(col)
#             continue
        
#         # Fill missing values based on data type
#         if df_copy[col].dtype in ['float32', 'float64', 'int32', 'int64']:
#             # For numeric columns, use median
#             fill_value = df_copy[col].median()
#             df_copy[col] = df_copy[col].fillna(fill_value)
#         else:
#             # For non-numeric columns, use mode or 0
#             if df_copy[col].dtype == 'object':
#                 # For categorical/string columns, use mode
#                 mode_value = df_copy[col].mode()
#                 if len(mode_value) > 0:
#                     df_copy[col] = df_copy[col].fillna(mode_value[0])
#                 else:
#                     df_copy[col] = df_copy[col].fillna('unknown')
#             else:
#                 # For other types, use 0
#                 df_copy[col] = df_copy[col].fillna(0)
    
#     # Drop columns with too many missing values
#     if cols_to_drop:
#         df_copy = df_copy.drop(columns=cols_to_drop)
#         print(f"Dropped {len(cols_to_drop)} columns with ≥{missing_threshold:.1%} missing values")
    
#     return df_copy

# train_df = handle_missing_values(train_df)
# test_df = handle_missing_values(test_df)



In [ ]:
def handle_missing_values_safe(train_df: pd.DataFrame, test_df: pd.DataFrame, 
                               missing_threshold: float = 0.95) -> tuple[pd.DataFrame, pd.DataFrame]:
    """
    Handle missing values in train and test DataFrames, only dropping columns that exist in both
    
    Parameters:
    - train_df: Training DataFrame
    - test_df: Test DataFrame  
    - missing_threshold: Drop columns with missing ratio >= this threshold (default: 0.95)
    
    Returns:
    - Tuple of (train_df, test_df) with missing values handled
    """
    train_copy = train_df.copy()
    test_copy = test_df.copy()
    
    # Get columns that exist in both datasets
    common_columns = set(train_copy.columns) & set(test_copy.columns)
    
    # Track columns to drop (only from common columns)
    cols_to_drop = []
    
    # Check each common column for missing values
    for col in common_columns:
        # Check missing ratio in both train and test
        train_missing_ratio = train_copy[col].isna().sum() / len(train_copy)
        test_missing_ratio = test_copy[col].isna().sum() / len(test_copy)
        
        # Drop if either dataset has too many missing values
        if train_missing_ratio >= missing_threshold or test_missing_ratio >= missing_threshold:
            cols_to_drop.append(col)
            print(f"Column '{col}': train missing {train_missing_ratio:.1%}, test missing {test_missing_ratio:.1%}")
    
    # Drop the identified columns from both datasets
    if cols_to_drop:
        train_copy = train_copy.drop(columns=cols_to_drop)
        test_copy = test_copy.drop(columns=cols_to_drop)
        print(f"Dropped {len(cols_to_drop)} common columns with ≥{missing_threshold:.1%} missing values")
    
    # Fill missing values in train dataset
    for col in train_copy.columns:
        missing_count = train_copy[col].isna().sum()
        if missing_count == 0:
            continue
            
        if train_copy[col].dtype in ['float32', 'float64', 'int32', 'int64']:
            fill_value = train_copy[col].median()
            train_copy[col] = train_copy[col].fillna(fill_value)
        elif train_copy[col].dtype == 'object':
            mode_value = train_copy[col].mode()
            if len(mode_value) > 0:
                train_copy[col] = train_copy[col].fillna(mode_value[0])
            else:
                train_copy[col] = train_copy[col].fillna('unknown')
        else:
            train_copy[col] = train_copy[col].fillna(0)
    
    # Fill missing values in test dataset
    for col in test_copy.columns:
        missing_count = test_copy[col].isna().sum()
        if missing_count == 0:
            continue
            
        if test_copy[col].dtype in ['float32', 'float64', 'int32', 'int64']:
            fill_value = test_copy[col].median()
            test_copy[col] = test_copy[col].fillna(fill_value)
        elif test_copy[col].dtype == 'object':
            mode_value = test_copy[col].mode()
            if len(mode_value) > 0:
                test_copy[col] = test_copy[col].fillna(mode_value[0])
            else:
                test_copy[col] = test_copy[col].fillna('unknown')
        else:
            test_copy[col] = test_copy[col].fillna(0)
    
    print("Missing value handling completed")
    return train_copy, test_copy

train_df, test_df = handle_missing_values_safe(train_df, test_df)
gc.collect()

In [ ]:
# def handle_missing_values(df: pd.DataFrame, missing_threshold: float = 0.95) -> pd.DataFrame:
#     """
#     Handle missing values in a DataFrame
    
#     Parameters:
#     - df: Input DataFrame
#     - missing_threshold: Drop columns with missing ratio >= this threshold (default: 0.95)
    
#     Returns:
#     - DataFrame with missing values handled
#     """
#     df_copy = df.copy()
    
#     # Get all columns (excluding non-feature columns if needed)
#     feature_cols = df_copy.columns.tolist()
    
#     # Track columns to drop
#     cols_to_drop = []
    
#     # Check each column for missing values
#     for col in feature_cols:
#         missing_count = df_copy[col].isna().sum()
#         missing_ratio = missing_count / len(df_copy)
        
#         # Drop columns with too many missing values
#         if missing_ratio >= missing_threshold:
#             cols_to_drop.append(col)
#             continue
        
#         # Fill missing values based on data type
#         if df_copy[col].dtype in ['float32', 'float64', 'int32', 'int64']:
#             # For numeric columns, use median
#             fill_value = df_copy[col].median()
#             df_copy[col] = df_copy[col].fillna(fill_value)
#         else:
#             # For non-numeric columns, use mode or 0
#             if df_copy[col].dtype == 'object':
#                 # For categorical/string columns, use mode
#                 mode_value = df_copy[col].mode()
#                 if len(mode_value) > 0:
#                     df_copy[col] = df_copy[col].fillna(mode_value[0])
#                 else:
#                     df_copy[col] = df_copy[col].fillna('unknown')
#             else:
#                 # For other types, use 0
#                 df_copy[col] = df_copy[col].fillna(0)
    
#     # Drop columns with too many missing values
#     if cols_to_drop:
#         df_copy = df_copy.drop(columns=cols_to_drop)
#         print(f"Dropped {len(cols_to_drop)} columns with ≥{missing_threshold:.1%} missing values")
    
#     return df_copy

# df_event = handle_missing_values(df_event)
# df_trans = handle_missing_values(df_trans)
# df_offer = handle_missing_values(df_offer)
# gc.collect()


In [ ]:
# import pandas as pd
# import numpy as np
# from sklearn.preprocessing import StandardScaler
# from sklearn.cluster import KMeans

# def add_new_features(train_df: pd.DataFrame, test_df: pd.DataFrame) -> tuple[pd.DataFrame, pd.DataFrame]:
#     """
#     Add new features to train and test DataFrames including interaction, customer-level aggregations,
#     temporal, clustering, ratio, offer frequency, and cyclical encoding for timestamp features.
#     Avoids using the target variable 'y' to prevent target leakage.

#     Parameters:
#     - train_df: Training DataFrame with 'id4' (datetime or convertible to datetime)
#     - test_df: Test DataFrame with 'id4' (datetime or convertible to datetime)

#     Returns:
#     - Tuple of (train_df, test_df) with new features added
#     """
#     # Create copies to avoid modifying original DataFrames
#     train_df = train_df.copy()
#     test_df = test_df.copy()

#     # Check for required columns
#     required_cols = ['id2', 'id3', 'id4', 'f132', 'best_day_ctr', 'f366', 'overall_ctr', 'performance_consistency', 'engagement_intensity']
#     missing_cols_train = [col for col in required_cols + ['y'] if col not in train_df.columns]
#     if missing_cols_train:
#         raise KeyError(f"Missing columns in train_df: {missing_cols_train}")
#     missing_cols_test = [col for col in required_cols if col not in test_df.columns]
#     if missing_cols_test:
#         raise KeyError(f"Missing columns in test_df: {missing_cols_test}")

#     # Convert id4 to datetime and handle invalid/missing values
#     print("Converting id4 to datetime...")
#     for df, name in [(train_df, 'train_df'), (test_df, 'test_df')]:
#         try:
#             df['id4'] = pd.to_datetime(df['id4'], errors='coerce')
#             if df['id4'].isna().any():
#                 print(f"Warning: {name} has {df['id4'].isna().sum()} invalid id4 values. Filling with max timestamp.")
#                 df['id4'] = df['id4'].fillna(df['id4'].max())
#             print(f"{name} id4 dtype: {df['id4'].dtype}")
#         except Exception as e:
#             raise ValueError(f"Failed to convert id4 to datetime in {name}: {str(e)}")

#     # 1. Interaction Features
#     print("Adding interaction features...")
#     for df, name in [(train_df, 'train_df'), (test_df, 'test_df')]:
#         df['f132_best_day_ctr'] = df['f132'] * df['best_day_ctr']
#         print(f"Created feature for {name}: f132_best_day_ctr")
#         df['f366_over_overall_ctr'] = df['f366'] / (df['overall_ctr'] + 1e-10)
#         print(f"Created feature for {name}: f366_over_overall_ctr")

#     # 2. Customer-Level Aggregations
#     print("Adding customer-level aggregation features...")
#     for df, name in [(train_df, 'train_df'), (test_df, 'test_df')]:
#         df['mean_f132_per_id2'] = df.groupby('id2')['f132'].transform('mean')
#         print(f"Created feature for {name}: mean_f132_per_id2")
#         df['max_best_day_ctr_per_id2'] = df.groupby('id2')['best_day_ctr'].transform('max')
#         print(f"Created feature for {name}: max_best_day_ctr_per_id2")

#     # 3. Temporal Features
#     print("Adding temporal features...")
#     for df, name in [(train_df, 'train_df'), (test_df, 'test_df')]:
#         df['recency'] = df.groupby('id2')['id4'].transform(lambda x: (x.max() - x).dt.total_seconds() / 3600)
#         print(f"Created feature for {name}: recency")

#     # Additional temporal features
#     for df, name in [(train_df, 'train_df'), (test_df, 'test_df')]:
#         if df is not None:
#             # Time since previous transaction in hours per customer
#             df['time_since_prev_txn_hours'] = df.groupby('id2')['id4'].diff().dt.total_seconds() / 3600.0
#             df['time_since_prev_txn_hours'] = df['time_since_prev_txn_hours'].fillna(df['time_since_prev_txn_hours'].max())
#             print(f"Created feature for {name}: time_since_prev_txn_hours")
            
#             # Rolling average of f132 (3-transaction window)
#             df['f132_rolling_mean_3'] = df.groupby('id2')['f132'].transform(lambda x: x.rolling(window=3, min_periods=1).mean())
#             print(f"Created feature for {name}: f132_rolling_mean_3")
            
#             # Standard deviation of best_day_ctr per customer
#             df['best_day_ctr_std_per_id2'] = df.groupby('id2')['best_day_ctr'].transform('std').fillna(0)
#             print(f"Created feature for {name}: best_day_ctr_std_per_id2")

#     # Non-optimized rolling count function (reverted to original)
#     def add_rolling_counts(df, name, hours):
#         """Add rolling count of transactions for specified hours"""
#         df[f'transaction_count_last_{hours}h'] = 0
#         for id2 in df['id2'].unique():
#             customer_mask = df['id2'] == id2
#             customer_data = df[customer_mask].copy()
#             customer_data = customer_data.sort_values('id4')
#             rolling_counts = []
#             for current_time in customer_data['id4']:
#                 time_window = current_time - pd.Timedelta(hours=hours)
#                 count = ((customer_data['id4'] >= time_window) & (customer_data['id4'] <= current_time)).sum()
#                 rolling_counts.append(count)
#             df.loc[customer_mask, f'transaction_count_last_{hours}h'] = rolling_counts
#         # Debugging: Log summary statistics for rolling counts
#         print(f"{name} transaction_count_last_{hours}h stats: mean={df[f'transaction_count_last_{hours}h'].mean():.2f}, "
#               f"min={df[f'transaction_count_last_{hours}h'].min()}, "
#               f"max={df[f'transaction_count_last_{hours}h'].max()}")
#         return df

#     # Apply rolling counts for 6h, 12h, 24h
#     for df, name in [(train_df, 'train_df'), (test_df, 'test_df')]:
#         df = add_rolling_counts(df, name, 6)
#         df = add_rolling_counts(df, name, 12)
#         df = add_rolling_counts(df, name, 24)

#     def time_since_last_event(df, name):
#         df['time_since_last_event'] = df.groupby('id2')['id4'].diff().shift(-1).dt.total_seconds() / 3600
#         df['time_since_last_event'] = df['time_since_last_event'].fillna(df['time_since_last_event'].max())
#         print(f"Created feature for {name}: time_since_last_event")
#         # Debugging: Log summary statistics
#         print(f"{name} time_since_last_event stats: mean={df['time_since_last_event'].mean():.2f}, "
#               f"min={df['time_since_last_event'].min():.2f}, "
#               f"max={df['time_since_last_event'].max():.2f}")
#         return df

#     train_df = time_since_last_event(train_df, 'train_df')
#     test_df = time_since_last_event(test_df, 'test_df')

#     # 4. Cyclical Encoding for Timestamp Features
#     print("Adding cyclical encoding for timestamp features...")
#     def cyclical_encode(df, name, column, period):
#         df[f'{column}_sin'] = np.sin(2 * np.pi * df[column] / period)
#         df[f'{column}_cos'] = np.cos(2 * np.pi * df[column] / period)
#         print(f"Created features for {name}: {column}_sin, {column}_cos")
#         return df

#     # Extract time components from id4
#     for df, name in [(train_df, 'train_df'), (test_df, 'test_df')]:
#         df['hour'] = df['id4'].dt.hour
#         df['day_of_week'] = df['id4'].dt.dayofweek
#         df['month'] = df['id4'].dt.month
#         # Apply cyclical encoding
#         df = cyclical_encode(df, name, 'hour', 24)
#         df = cyclical_encode(df, name, 'day_of_week', 7)
#         df = cyclical_encode(df, name, 'month', 12)
#         # Drop original time components to avoid redundancy
#         df = df.drop(['hour', 'day_of_week', 'month'], axis=1)

#     # 5. Feature Clustering
#     print("Adding clustering features...")
#     cluster_features = ['f132', 'best_day_ctr']
#     scaler = StandardScaler()
#     cluster_data = scaler.fit_transform(train_df[cluster_features].fillna(0))
#     kmeans = KMeans(n_clusters=10, random_state=42)
#     train_df['customer_cluster'] = kmeans.fit_predict(cluster_data)
#     print("Created feature for train_df: customer_cluster")
#     test_df['customer_cluster'] = kmeans.predict(scaler.transform(test_df[cluster_features].fillna(0)))
#     print("Created feature for test_df: customer_cluster")

#     # 6. Ratio Features
#     print("Adding ratio features...")
#     for df, name in [(train_df, 'train_df'), (test_df, 'test_df')]:
#         df['perf_consistency_over_engage'] = df['performance_consistency'] / (df['engagement_intensity'] + 1e-10)
#         print(f"Created feature for {name}: perf_consistency_over_engage")

#     # 7. Offer Frequency (replacing offer_click_rate to avoid target leakage)
#     print("Adding offer frequency feature...")
#     offer_frequency = train_df.groupby('id3').size().to_dict()
#     train_df['offer_frequency'] = train_df['id3'].map(offer_frequency)
#     print("Created feature for train_df: offer_frequency")
#     test_df['offer_frequency'] = test_df['id3'].map(offer_frequency)
#     print("Created feature for test_df: offer_frequency")

#     # 8. New Temporal Features
#     print("Adding new temporal features...")
#     for df, name in [(train_df, 'train_df'), (test_df, 'test_df')]:
#         # 1. Log-transform of time_since_last_event
#         df['log_time_since_last_event'] = np.log1p(df['time_since_last_event'].clip(lower=0))
#         print(f"Created feature for {name}: log_time_since_last_event")
        
#         # 2. Time since first event per id2 (in hours)
#         first_event = df.groupby('id2')['id4'].transform('min')
#         df['time_since_first_event_hours'] = (df['id4'] - first_event).dt.total_seconds() / 3600.0
#         df['time_since_first_event_hours'] = df['time_since_first_event_hours'].fillna(df['time_since_first_event_hours'].max())
#         print(f"Created feature for {name}: time_since_first_event_hours")
        
#         # 5. Hour of day (non-cyclical, for compatibility with existing features)
#         df['hour_of_day'] = df['id4'].dt.hour
#         print(f"Created feature for {name}: hour_of_day")
        
#         # 6. Day of week (non-cyclical, for compatibility)
#         df['day_of_week'] = df['id4'].dt.dayofweek
#         print(f"Created feature for {name}: day_of_week")
        
#         # 7. Rolling std of f132 over last 3 transactions
#         df['f132_rolling_std_3'] = df.groupby('id2')['f132'].transform(
#             lambda x: x.rolling(window=3, min_periods=1).std()
#         ).fillna(0)
#         print(f"Created feature for {name}: f132_rolling_std_3")
        
#         # 8. Ratio of f132_best_day_ctr to mean_f132_per_id2
#         df['f132_best_day_ctr_ratio'] = df['f132_best_day_ctr'] / (df['mean_f132_per_id2'] + 1e-10)
#         print(f"Created feature for {name}: f132_best_day_ctr_ratio")
        
#         # 9. Difference between f210 and f366
#         if 'f210' in df.columns:
#             df['f210_f366_diff'] = df['f210'] - df['f366']
#         else:
#             df['f210_f366_diff'] = 0  # Placeholder if f210 is missing
#         print(f"Created feature for {name}: f210_f366_diff")
        
#         # 10. Transaction velocity (transactions per hour)
#         df['transaction_velocity_12h'] = df['transaction_count_last_12h'] / (df['time_since_prev_txn_hours'] + 1e-10)
#         print(f"Created feature for {name}: transaction_velocity_12h")
#         # Debugging: Log summary statistics
#         print(f"{name} transaction_velocity_12h stats: mean={df['transaction_velocity_12h'].mean():.2f}, "
#               f"min={df['transaction_velocity_12h'].min():.2f}, "
#               f"max={df['transaction_velocity_12h'].max():.2f}")

#     # List of new features
#     new_features = [
#         'f132_best_day_ctr', 'f366_over_overall_ctr', 'mean_f132_per_id2',
#         'max_best_day_ctr_per_id2', 'recency', 'time_since_prev_txn_hours',
#         'f132_rolling_mean_3', 'best_day_ctr_std_per_id2', 'transaction_count_last_12h',
#         'time_since_last_event', 'hour_sin', 'hour_cos', 'day_of_week_sin', 'day_of_week_cos',
#         'month_sin', 'month_cos', 'customer_cluster', 'perf_consistency_over_engage',
#         'offer_frequency', 'log_time_since_last_event', 'time_since_first_event_hours',
#         'transaction_count_last_6h', 'transaction_count_last_24h', 'hour_of_day',
#         'day_of_week', 'f132_rolling_std_3', 'f132_best_day_ctr_ratio',
#         'f210_f366_diff', 'transaction_velocity_12h'
#     ]
#     print("Feature engineering completed. Added features:")
#     print(new_features)

#     return train_df, test_df


In [ ]:
train_df = train_df.sort_values('id4', ascending=True)

In [ ]:
def train_val_split(df: pd.DataFrame, val_ratio: float = 0.3) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Split sorted DataFrame into train and validation sets
    
    Parameters:
    - df: Input sorted DataFrame
    - val_ratio: Ratio of data to use for validation (default: 0.2 for 20%)
    
    Returns:
    - Tuple of (train_df, val_df)
    """
    total_rows = len(df)
    train_size = int(total_rows * (1 - val_ratio))
    
    # Top 80% for training
    train_df = df.iloc[:train_size].copy()
    
    # Bottom 20% for validation
    val_df = df.iloc[train_size:].copy()
    
    print(f"Total rows: {total_rows:,}")
    print(f"Train rows: {len(train_df):,} ({len(train_df)/total_rows:.1%})")
    print(f"Val rows: {len(val_df):,} ({len(val_df)/total_rows:.1%})")
    
    return train_df, val_df

train_df, val_df = train_val_split(train_df)
gc.collect()

In [ ]:
val_df.head()

In [ ]:
len(val_df['id2'].unique())

In [ ]:
# exclude_cols = ['id1', 'id3', 'id4', 'id5', 'id6', 'id7', 'id8', 
#                    'id9', 'id10', 'id11', 'id12', 'id13', 'pred']
# exclude_cols.extend([col for col in train_df.columns if col.endswith('_dt')])

In [ ]:
# def remove_excluded_columns(train_df: pd.DataFrame, val_df: pd.DataFrame, 
#                           exclude_cols: List[str]) -> Tuple[pd.DataFrame, pd.DataFrame]:
#     """
#     Remove excluded columns from train and validation DataFrames
    
#     Parameters:
#     - train_df: Training DataFrame
#     - val_df: Validation DataFrame  
#     - exclude_cols: List of column names to exclude
    
#     Returns:
#     - Tuple of (cleaned_train_df, cleaned_val_df)
#     """
#     # Find columns that actually exist in the DataFrames
#     train_cols_to_drop = [col for col in exclude_cols if col in train_df.columns]
#     val_cols_to_drop = [col for col in exclude_cols if col in val_df.columns]
    
#     # Drop columns
#     train_df_clean = train_df.drop(columns=train_cols_to_drop)
#     val_df_clean = val_df.drop(columns=val_cols_to_drop)
    
#     print(f"Dropped {len(train_cols_to_drop)} columns from train_df")
#     print(f"Dropped {len(val_cols_to_drop)} columns from val_df")
    
#     if train_cols_to_drop:
#         print(f"Columns dropped from train: {train_cols_to_drop}")
#     if val_cols_to_drop:
#         print(f"Columns dropped from val: {val_cols_to_drop}")
    
#     return train_df_clean, val_df_clean

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from numba import njit
import numpy as np

# Numba function for rolling count in 12-hour window

@njit
def rolling_count_numba(timestamps_seconds, window_size_hours=12):
    n = len(timestamps_seconds)
    result = np.zeros(n, dtype=np.int32)
    window_size_seconds = window_size_hours * 3600.0

    for i in range(n):
        current_time = timestamps_seconds[i]
        count = 0
        for j in range(n):
            if timestamps_seconds[j] >= current_time - window_size_seconds and timestamps_seconds[j] <= current_time:
                count += 1
        result[i] = count
    return result

def rolling_count_12h_numba(group):
    group_sorted = group.sort_values('id4')
    timestamps = group_sorted['id4'].astype(np.int64).to_numpy() / 1e9  # Fix: convert to NumPy array
    counts = rolling_count_numba(timestamps)
    return pd.Series(counts, index=group_sorted.index)



def add_new_features(train_df: pd.DataFrame, val_df: pd.DataFrame, test_df: pd.DataFrame) -> tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """
    Add new features to train, validation, and test DataFrames including interaction, customer-level aggregations,
    temporal, clustering, ratio, offer frequency, and cyclical encoding for timestamp features.
    Avoids using the target variable 'y' to prevent target leakage.

    Parameters:
    - train_df: Training DataFrame with 'id4' (datetime or convertible to datetime)
    - val_df: Validation DataFrame with 'id4' (datetime or convertible to datetime)
    - test_df: Test DataFrame with 'id4' (datetime or convertible to datetime)

    Returns:
    - Tuple of (train_df, val_df, test_df) with new features added
    """
    # Create copies to avoid modifying original DataFrames
    train_df = train_df.copy()
    val_df = val_df.copy()
    test_df = test_df.copy()

    # Check for required columns
    required_cols = ['id2', 'id3', 'id4', 'f132', 'best_day_ctr', 'f366', 'overall_ctr', 'performance_consistency', 'engagement_intensity']
    # Training and validation require 'y', but test may not
    for df, name in [(train_df, 'train_df'), (val_df, 'val_df')]:
        missing_cols = [col for col in required_cols + ['y'] if col not in df.columns]
        if missing_cols:
            raise KeyError(f"Missing columns in {name}: {missing_cols}")
    # Test DataFrame may not have 'y'
    missing_cols_test = [col for col in required_cols if col not in test_df.columns]
    if missing_cols_test:
        raise KeyError(f"Missing columns in test_df: {missing_cols_test}")

    # Convert id4 to datetime and handle invalid/missing values
    print("Converting id4 to datetime...")
    for df, name in [(train_df, 'train_df'), (val_df, 'val_df'), (test_df, 'test_df')]:
        try:
            df['id4'] = pd.to_datetime(df['id4'], errors='coerce')
            if df['id4'].isna().any():
                print(f"Warning: {name} has {df['id4'].isna().sum()} invalid id4 values. Filling with max timestamp.")
                df['id4'] = df['id4'].fillna(df['id4'].max())
            print(f"{name} id4 dtype: {df['id4'].dtype}")
        except Exception as e:
            raise ValueError(f"Failed to convert id4 to datetime in {name}: {str(e)}")

    # 1. Interaction Features
    print("Adding interaction features...")
    for df in [train_df, val_df, test_df]:
        df['f132_best_day_ctr'] = df['f132'] * df['best_day_ctr']
        df['f366_over_overall_ctr'] = df['f366'] / (df['overall_ctr'] + 1e-10)

    # 2. Customer-Level Aggregations
    print("Adding customer-level aggregation features...")
    for df in [train_df, val_df, test_df]:
        df['mean_f132_per_id2'] = df.groupby('id2')['f132'].transform('mean')
        df['max_best_day_ctr_per_id2'] = df.groupby('id2')['best_day_ctr'].transform('max')

    # 3. Temporal Features
    print("Adding temporal features...")
    for df in [train_df, val_df, test_df]:
        df['recency'] = df.groupby('id2')['id4'].transform(lambda x: (x.max() - x).dt.total_seconds() / 3600)

    for df in [train_df, val_df, test_df]:
        if df is not None:
            df_sorted = df.sort_values(['id2', 'id4'])
            
            # Time since previous transaction
            df['time_since_prev_txn_hours'] = df_sorted.groupby('id2')['id4'].diff().dt.total_seconds() / 3600.0
            df['time_since_prev_txn_hours'] = df['time_since_prev_txn_hours'].fillna(df['time_since_prev_txn_hours'].max())
            
            # Rolling average of f132
            df['f132_rolling_mean_3'] = df_sorted.groupby('id2')['f132'].transform(lambda x: x.rolling(window=3, min_periods=1).mean())
            
            # Replace slow rolling count with Numba version
            df['transaction_count_last_12h'] = df.groupby('id2').apply(rolling_count_12h_numba).reset_index(level=0, drop=True).reindex(df.index)
            
            # Std deviation of best_day_ctr
            df['best_day_ctr_std_per_id2'] = df.groupby('id2')['best_day_ctr'].transform('std').fillna(0)


    def time_since_last_event(df):
        df = df.sort_values(['id2', 'id4'])
        df['time_since_last_event'] = df.groupby('id2')['id4'].diff().shift(-1).dt.total_seconds() / 3600
        df['time_since_last_event'] = df['time_since_last_event'].fillna(df['time_since_last_event'].max())
        return df

    train_df = time_since_last_event(train_df)
    val_df = time_since_last_event(val_df)
    test_df = time_since_last_event(test_df)

    # 4. Cyclical Encoding for Timestamp Features
    print("Adding cyclical encoding for timestamp features...")
    def cyclical_encode(df, column, period):
        df[f'{column}_sin'] = np.sin(2 * np.pi * df[column] / period)
        df[f'{column}_cos'] = np.cos(2 * np.pi * df[column] / period)
        return df

    # Extract time components from id4
    for df in [train_df, val_df, test_df]:
        df['hour'] = df['id4'].dt.hour
        df['day_of_week'] = df['id4'].dt.dayofweek
        df['month'] = df['id4'].dt.month
        # Apply cyclical encoding
        df = cyclical_encode(df, 'hour', 24)
        df = cyclical_encode(df, 'day_of_week', 7)
        df = cyclical_encode(df, 'month', 12)
        # Drop original time components to avoid redundancy
        df = df.drop(['hour', 'day_of_week', 'month'], axis=1)

    # 5. Feature Clustering
    print("Adding clustering features...")
    cluster_features = ['f132', 'best_day_ctr']
    scaler = StandardScaler()
    cluster_data = scaler.fit_transform(train_df[cluster_features].fillna(0))
    kmeans = KMeans(n_clusters=10, random_state=42)
    train_df['customer_cluster'] = kmeans.fit_predict(cluster_data)
    val_df['customer_cluster'] = kmeans.predict(scaler.transform(val_df[cluster_features].fillna(0)))
    test_df['customer_cluster'] = kmeans.predict(scaler.transform(test_df[cluster_features].fillna(0)))

    # 6. Ratio Features
    print("Adding ratio features...")
    for df in [train_df, val_df, test_df]:
        df['perf_consistency_over_engage'] = df['performance_consistency'] / (df['engagement_intensity'] + 1e-10)

    # 7. Offer Frequency (replacing offer_click_rate to avoid target leakage)
    print("Adding offer frequency feature...")
    offer_frequency = train_df.groupby('id3').size().to_dict()
    train_df['offer_frequency'] = train_df['id3'].map(offer_frequency)
    val_df['offer_frequency'] = val_df['id3'].map(offer_frequency)
    test_df['offer_frequency'] = test_df['id3'].map(offer_frequency)

    # List of new features
    new_features = [
        'f132_best_day_ctr', 'f366_over_overall_ctr', 'mean_f132_per_id2',
        'max_best_day_ctr_per_id2', 'recency', 'time_since_prev_txn_hours',
        'f132_rolling_mean_3', 'transaction_count_last_12h', 'best_day_ctr_std_per_id2',
        'time_since_last_event', 'hour_sin', 'hour_cos', 'day_of_week_sin', 'day_of_week_cos',
        'month_sin', 'month_cos', 'customer_cluster', 'perf_consistency_over_engage',
        'offer_frequency'
    ]
    print("Feature engineering completed. Added features:")
    print(new_features)

    return train_df, val_df, test_df

In [ ]:
df_event.columns

In [ ]:
import pandas as pd
import numpy as np
import cupy as cp
import cudf
from cuml.preprocessing import StandardScaler as cuStandardScaler
from cuml.cluster import KMeans as cuKMeans
from cuml.decomposition import PCA as cuPCA
from scipy.stats import entropy
import warnings
warnings.filterwarnings('ignore')
from numba import cuda, jit, prange
import dask_cudf
from dask.distributed import Client
from dask_cuda import LocalCUDACluster

# ==========================================
# GPU UTILITIES
# ==========================================

def to_cudf(df):
    """Convert pandas DataFrame to cuDF DataFrame"""
    return cudf.from_pandas(df)

def to_pandas(gdf):
    """Convert cuDF DataFrame back to pandas"""
    return gdf.to_pandas()

@cuda.jit
def calculate_ctr_kernel(impressions, clicks, ctr_out):
    """CUDA kernel for CTR calculation"""
    idx = cuda.grid(1)
    if idx < impressions.size:
        ctr_out[idx] = clicks[idx] / (impressions[idx] + 1e-6)

# ==========================================
# 1. GPU-OPTIMIZED BEHAVIORAL PROXIES
# ==========================================

def identify_overlap_customers(df_event, df_trans):
    """GPU-accelerated customer overlap identification"""
    # Convert to GPU
    gdf_event = to_cudf(df_event[['id2']].drop_duplicates())
    gdf_trans = to_cudf(df_trans[['id2']].drop_duplicates())
    
    # GPU merge to find overlaps
    overlap = gdf_event.merge(gdf_trans, on='id2', how='inner')
    overlap_customers = overlap['id2'].to_pandas().tolist()
    
    print(f"Found {len(overlap_customers)} overlapping customers")
    return overlap_customers

def calculate_same_day_activity_gpu(gdf_trans, gdf_event, profiles):
    """GPU-accelerated same-day activity calculation"""
    # Extract dates
    gdf_trans['trans_date'] = cudf.to_datetime(gdf_trans['f370']).dt.floor('D')
    gdf_event['event_date'] = cudf.to_datetime(gdf_event['id4']).dt.floor('D')
    
    # Count unique dates per customer
    trans_dates = gdf_trans.groupby('id2')['trans_date'].nunique().reset_index()
    trans_dates.columns = ['id2', 'trans_unique_days']
    
    event_dates = gdf_event.groupby('id2')['event_date'].nunique().reset_index()
    event_dates.columns = ['id2', 'event_unique_days']
    
    # Merge dates for same-day detection
    merged_dates = gdf_trans[['id2', 'trans_date']].merge(
        gdf_event[['id2', 'event_date']], 
        left_on=['id2', 'trans_date'], 
        right_on=['id2', 'event_date'],
        how='inner'
    )
    
    same_day_counts = merged_dates.groupby('id2').size().reset_index()
    same_day_counts.columns = ['id2', 'same_day_count']
    
    # Add to profiles
    profiles = profiles.merge(trans_dates, on='id2', how='left')
    profiles = profiles.merge(event_dates, on='id2', how='left')
    profiles = profiles.merge(same_day_counts, on='id2', how='left')
    
    # Calculate ratio
    total_unique_days = profiles['trans_unique_days'] + profiles['event_unique_days'] - profiles['same_day_count']
    profiles['same_day_activity_ratio'] = profiles['same_day_count'] / (total_unique_days + 1e-6)
    
    return profiles

def create_customer_profiles_gpu(df_event, df_trans, overlap_customers):
    """Vectorized GPU customer profiling"""
    print("Creating GPU-accelerated customer profiles...")
    
    # Convert to GPU DataFrames
    gdf_trans = to_cudf(df_trans[df_trans['id2'].isin(overlap_customers)])
    gdf_event = to_cudf(df_event[df_event['id2'].isin(overlap_customers)])
    
    # Transaction aggregations on GPU
    trans_agg = gdf_trans.groupby('id2').agg({
        'f367': ['mean', 'count', 'std', 'sum']
    }).reset_index()
    trans_agg.columns = ['id2', 'avg_transaction_amount', 'transaction_count', 
                        'transaction_std', 'total_trans_amount']
    
    # Event aggregations on GPU
    event_agg = gdf_event.groupby('id2').agg({
        'id3': 'count',
        'id7': 'count'
    }).reset_index()
    event_agg.columns = ['id2', 'impression_count', 'click_count']
    
    # Unique offers count
    unique_offers = gdf_event.groupby('id2')['id3'].nunique().reset_index()
    unique_offers.columns = ['id2', 'unique_offers_seen']
    
    # Merge all on GPU
    profiles = trans_agg.merge(event_agg, on='id2', how='outer')
    profiles = profiles.merge(unique_offers, on='id2', how='left')
    
    # Calculate derived features on GPU
    profiles['transaction_cv'] = profiles['transaction_std'] / (profiles['avg_transaction_amount'] + 1e-6)
    profiles['ctr'] = profiles['click_count'] / (profiles['impression_count'] + 1e-6)
    
    # Calculate same-day activity using GPU operations
    profiles = calculate_same_day_activity_gpu(gdf_trans, gdf_event, profiles)
    
    # Fill NaNs and convert back to pandas
    profiles = profiles.fillna(0)
    return profiles.to_pandas()

def create_behavioral_clusters_gpu(profiles_df, n_clusters=25):
    """GPU-accelerated clustering"""
    cluster_features = [
        'avg_transaction_amount', 'transaction_count', 'transaction_cv',
        'impression_count', 'ctr', 'same_day_activity_ratio'
    ]
    
    # Convert to GPU
    X = profiles_df[cluster_features].fillna(0).values
    X_gpu = cp.asarray(X)
    
    # Scale on GPU
    scaler = cuStandardScaler()
    X_scaled = scaler.fit_transform(X_gpu)
    
    # GPU KMeans
    kmeans = cuKMeans(
        n_clusters=n_clusters,
        random_state=42,
        n_init=10,
        max_iter=300
    )
    
    # Fit and predict
    labels = kmeans.fit_predict(X_scaled)
    profiles_df['behavior_cluster'] = cp.asnumpy(labels)
    
    # Calculate cluster profiles
    cluster_profiles = profiles_df.groupby('behavior_cluster')[cluster_features].mean()
    
    # Store profiles with index for fast lookup
    customer_clusters = profiles_df.set_index('id2')['behavior_cluster'].to_dict()
    
    return profiles_df, cluster_profiles, customer_clusters, scaler, kmeans

def batch_assign_clusters(customer_features_df, scaler, kmeans):
    """Batch assign clusters using GPU"""
    feature_cols = ['avg_transaction_amount', 'transaction_count', 'transaction_cv',
                   'impression_count', 'ctr', 'same_day_activity_ratio']
    
    X = customer_features_df[feature_cols].fillna(0).values
    X_gpu = cp.asarray(X)
    X_scaled = scaler.transform(X_gpu)
    
    labels = kmeans.predict(X_scaled)
    return cp.asnumpy(labels)

# ==========================================
# 2. GPU-OPTIMIZED TEMPORAL FEATURES
# ==========================================

def define_holidays():
    """Define major shopping holidays"""
    return {
        '2023-11-24': 'black_friday',
        '2023-11-27': 'cyber_monday',
        '2023-12-25': 'christmas',
        '2023-10-31': 'halloween',
        '2023-11-23': 'thanksgiving',
        '2023-01-01': 'new_year',
        '2023-11-11': 'singles_day'
    }

def batch_calculate_offer_overlap(df_trans, df_offers, customer_ids):
    """Vectorized offer-transaction overlap calculation"""
    # Convert to GPU
    gdf_trans = to_cudf(df_trans)
    gdf_trans['date'] = cudf.to_datetime(gdf_trans['f370'])
    
    results = []
    
    # Process in batches for memory efficiency
    batch_size = 10000
    for i in range(0, len(customer_ids), batch_size):
        batch_customers = customer_ids[i:i+batch_size]
        
        # Filter transactions for batch
        batch_trans = gdf_trans[gdf_trans['id2'].isin(batch_customers)]
        
        # Vectorized calculations for each offer
        for _, offer in df_offers.iterrows():
            offer_start = pd.to_datetime(offer['id12'])
            offer_end = pd.to_datetime(offer['id13'])
            
            # Transactions during offer
            mask = (batch_trans['date'] >= offer_start) & (batch_trans['date'] <= offer_end)
            during_offer = batch_trans[mask]
            
            # Group by customer
            during_offer['day_only'] = during_offer['date'].dt.floor('D')
            offer_stats = during_offer.groupby('id2').agg({
                'f367': ['count', 'sum'],
                'day_only': 'nunique'
            }).reset_index()
            
            # Calculate baseline (30 days before)
            baseline_mask = (batch_trans['date'] >= offer_start - pd.Timedelta(days=30)) & \
                           (batch_trans['date'] < offer_start)
            baseline = batch_trans[baseline_mask]
            
            baseline_stats = baseline.groupby('id2').agg({
                'f367': 'count'
            }).reset_index()
            
            # Merge and calculate features
            merged = cudf.DataFrame({'id2': batch_customers})
            merged = merged.merge(offer_stats, on='id2', how='left')
            merged = merged.merge(baseline_stats, on='id2', how='left', suffixes=('', '_baseline'))
            
            # Calculate derived features
            merged['offer_id'] = offer['id3']
            merged['trans_count_during_offer'] = merged[('f367', 'count')].fillna(0)
            merged['trans_amount_during_offer'] = merged[('f367', 'sum')].fillna(0)
            merged['baseline_trans_rate'] = merged['f367_baseline'].fillna(0) / 30
            merged['offer_period_lift'] = merged['trans_count_during_offer'] / \
                                        (merged['baseline_trans_rate'] + 1e-6)
            
            results.append(merged.to_pandas())
    
    return pd.concat(results, ignore_index=True)

def vectorized_holiday_features(dates):
    """Vectorized holiday feature calculation using GPU"""
    holidays = define_holidays()
    holiday_dates = pd.to_datetime(list(holidays.keys()))
    
    dates_gpu = cp.asarray(pd.to_datetime(dates).values.astype('datetime64[D]').astype(int))
    holidays_gpu = cp.asarray(holiday_dates.values.astype('datetime64[D]').astype(int))
    
    # Calculate distance to each holiday for all dates at once
    distances = cp.abs(dates_gpu[:, None] - holidays_gpu[None, :])
    
    # Find nearest holiday
    min_distances = cp.min(distances, axis=1)
    nearest_holiday_idx = cp.argmin(distances, axis=1)
    
    # Convert back to pandas for feature creation
    features_df = pd.DataFrame({
        'days_to_nearest_holiday': cp.asnumpy(min_distances),
        'nearest_holiday_idx': cp.asnumpy(nearest_holiday_idx),
        'is_holiday_week': cp.asnumpy(min_distances <= 7)
    })
    
    # Map holiday types
    holiday_types = list(holidays.values())
    features_df['holiday_type'] = features_df['nearest_holiday_idx'].map(
        lambda x: holiday_types[x] if x < len(holiday_types) else 'none'
    )
    
    return features_df

# ==========================================
# 3. GPU-OPTIMIZED PROBABILISTIC MATCHING
# ==========================================

def build_industry_propensity_gpu(df_event, df_trans, df_offer):
    """GPU-accelerated industry propensity calculation"""
    # Convert to GPU
    gdf_event = to_cudf(df_event)
    gdf_trans = to_cudf(df_trans)
    gdf_offer = to_cudf(df_offer)
    
    # Get overlap customers
    overlap = gdf_event['id2'].unique().to_pandas()
    overlap = set(overlap) & set(gdf_trans['id2'].unique().to_pandas())
    overlap = list(overlap)[:50000]  # Limit for memory
    
    # Get customer industries
    counts = gdf_trans.groupby(['id2', 'id8']).size().reset_index(name='counts')
    cust_industries = counts.sort_values(['id2', 'counts'], ascending=[True, False]).drop_duplicates('id2')[['id2', 'id8']].reset_index()
    
    # Join events with offers
    event_offer = gdf_event[gdf_event['id2'].isin(overlap)].merge(
        gdf_offer[['id3', 'id10']], on='id3', how='left'
    )
    
    # Join with customer industries
    event_offer = event_offer.merge(cust_industries, on='id2', how='left')
    
    # Calculate CTR by industry pair
    clicks = event_offer.groupby(['id8', 'id10'])['id7'].count().reset_index(name='clicks')
    impressions = event_offer.groupby(['id8', 'id10']).size().reset_index(name='impressions')
    
    industry_stats = clicks.merge(impressions, on=['id8', 'id10'])
    industry_stats.columns = ['customer_industry', 'offer_industry', 'clicks', 'impressions']
    
    # Bayesian smoothing
    alpha, beta = 1, 10
    industry_stats['smoothed_ctr'] = (industry_stats['clicks'] + alpha) / \
                                    (industry_stats['impressions'] + beta)
    
    # Convert to lookup dictionary
    stats_pd = industry_stats.to_pandas()
    industry_propensity = {
        (row['customer_industry'], row['offer_industry']): row['smoothed_ctr']
        for _, row in stats_pd.iterrows()
    }
    
    default_ctr = (stats_pd['clicks'].sum() + alpha) / \
                  (stats_pd['impressions'].sum() + beta)
    
    return industry_propensity, default_ctr

def create_behavioral_archetypes_gpu(df_event, df_trans, n_archetypes=15):
    """GPU-accelerated archetype creation"""
    # Get overlap customers
    overlap = set(df_event['id2'].unique()) & set(df_trans['id2'].unique())
    sample_customers = list(overlap)[:20000]
    
    # Create feature matrix on GPU
    features_list = []
    
    # Process in batches
    batch_size = 5000
    for i in range(0, len(sample_customers), batch_size):
        batch = sample_customers[i:i+batch_size]
        
        # Convert to GPU
        batch_trans = to_cudf(df_trans[df_trans['id2'].isin(batch)])
        batch_event = to_cudf(df_event[df_event['id2'].isin(batch)])
        
        # Aggregate features
        trans_features = batch_trans.groupby('id2').agg({
            'f367': ['mean', 'std', 'count']
        }).reset_index()
        trans_features.columns = ['id2', 'f367_mean', 'f367_std', 'f367_count']
        
        clicks = batch_event.groupby('id2')['id7'].count().reset_index()
        clicks.columns = ['id2', 'clicks']
        
        impressions = batch_event.groupby('id2')['id3'].count().reset_index()
        impressions.columns = ['id2', 'impressions']
        
        event_features = clicks.merge(impressions, on='id2', how='outer')
        
        # Merge
        features = trans_features.merge(event_features, on='id2', how='outer')
        features_list.append(features.to_pandas())
    
    # Combine all features
    all_features = pd.concat(features_list, ignore_index=True)
    
    # Normalize and cluster on GPU
    feature_cols = all_features.columns[1:]
    X = all_features[feature_cols].fillna(0).values
    X_gpu = cp.asarray(X)
    
    # Scale
    scaler = cuStandardScaler()
    X_scaled = scaler.fit_transform(X_gpu)
    
    # Cluster
    kmeans = cuKMeans(n_clusters=n_archetypes, random_state=42)
    labels = kmeans.fit_predict(X_scaled)
    
    all_features['archetype'] = cp.asnumpy(labels)
    
    # Calculate archetype profiles
    archetype_profiles = all_features.groupby('archetype')[feature_cols].mean()
    
    # Calculate archetype CTRs
    all_features['ctr'] = all_features['clicks'] / (all_features['impressions'] + 1e-6)
    archetype_ctr = all_features.groupby('archetype')['ctr'].mean()
    
    return archetype_profiles, archetype_ctr, scaler, kmeans

def create_features_gpu_batch(df_data, df_event, df_trans, df_offer,
                             customer_clusters, cluster_profiles,
                             industry_propensity, default_ctr,
                             batch_size=50000):
    """Create features in GPU batches"""
    
    n_rows = len(df_data)
    all_features = []
    
    # Pre-compute expensive lookups
    print("   Pre-computing lookup tables...")
    
    # Get all unique customers and their transaction/event data
    unique_customers = df_data['id2'].unique()
    
    # Pre-aggregate customer data
    print("   Aggregating customer transaction data...")
    cust_trans_agg = df_trans.groupby('id2').agg({
        'f367': ['mean', 'std', 'count', 'sum'],
        'id8': lambda x: x.mode()[0] if len(x.mode()) > 0 else x.iloc[0]
    }).reset_index()
    cust_trans_agg.columns = ['id2', 'avg_trans', 'std_trans', 'count_trans', 'sum_trans', 'primary_industry']
    
    print("   Processing in batches...")
    for start_idx in range(0, n_rows, batch_size):
        end_idx = min(start_idx + batch_size, n_rows)
        print(f"   Batch {start_idx//batch_size + 1}/{(n_rows//batch_size) + 1}")
        
        batch_data = df_data.iloc[start_idx:end_idx].copy()
        
        # Merge pre-aggregated data
        batch_data = batch_data.merge(cust_trans_agg, on='id2', how='left')
        
        # Create features
        batch_features = pd.DataFrame(index=batch_data.index)
        
        # 1. Temporal features (vectorized)
        holiday_features = vectorized_holiday_features(batch_data['id5'])
        for col in holiday_features.columns:
            batch_features[f'temporal_{col}'] = holiday_features[col]
        
        # 2. Behavioral features
        # Check if customer is in overlap
        batch_features['behavioral_cluster'] = batch_data['id2'].map(
            customer_clusters
        ).fillna(-1)
        
        # Get cluster averages
        cluster_map = cluster_profiles['ctr'].to_dict()
        batch_features['behavioral_cluster_ctr'] = batch_features['behavioral_cluster'].map(
            cluster_map
        ).fillna(0)
        
        # 3. Probabilistic features
        # Get offer industries
        offer_industries = df_offer.set_index('id3')['id10'].to_dict()
        batch_data['offer_industry'] = batch_data['id3'].map(offer_industries)
        
        # Build tuple keys
        batch_keys = list(zip(batch_data['primary_industry'], batch_data['offer_industry']))
        batch_keys_series = pd.Series(batch_keys)
        
        # Vectorized lookup with .map()
        batch_features['prob_industry_propensity'] = batch_keys_series.map(
            lambda k: industry_propensity.get(k, default_ctr)
        )
        
        # Add basic stats as features
        batch_features['customer_avg_trans'] = batch_data['avg_trans'].fillna(0)
        batch_features['customer_trans_count'] = batch_data['count_trans'].fillna(0)
        batch_features['customer_trans_cv'] = (batch_data['std_trans'] / 
                                               (batch_data['avg_trans'] + 1e-6)).fillna(0)
        
        all_features.append(batch_features)
    
    return pd.concat(all_features, ignore_index=True)

# ==========================================
# MAIN GPU PIPELINE FUNCTION
# ==========================================

def add_gpu_features(train_df: pd.DataFrame, val_df: pd.DataFrame, test_df: pd.DataFrame,
                     df_event: pd.DataFrame, df_trans: pd.DataFrame, df_offer: pd.DataFrame,
                     sample_size: int = None, batch_size: int = 50000, 
                     n_clusters: int = 25) -> tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """
    Add GPU-accelerated behavioral, temporal, and probabilistic features to train, validation, and test DataFrames.
    
    Parameters:
    - train_df: Training DataFrame
    - val_df: Validation DataFrame  
    - test_df: Test DataFrame
    - df_event: Event data DataFrame
    - df_trans: Transaction data DataFrame
    - df_offer: Offer data DataFrame
    - sample_size: Optional sample size for faster processing
    - batch_size: Batch size for GPU processing
    - n_clusters: Number of behavioral clusters
    
    Returns:
    - Tuple of (train_df, val_df, test_df) with GPU-accelerated features added
    """
    print("=== Starting GPU-Accelerated Feature Pipeline ===")
    
    # Create copies to avoid modifying original DataFrames
    train_df = train_df.copy()
    val_df = val_df.copy()
    test_df = test_df.copy()
    
    # Sample if requested
    if sample_size:
        train_df = train_df.sample(n=sample_size, random_state=42)
        val_df = val_df.sample(n=min(sample_size//4, len(val_df)), random_state=42)
        test_df = test_df.sample(n=min(sample_size//4, len(test_df)), random_state=42)
    
    # Ensure data types
    for df in [train_df, val_df, test_df, df_event, df_trans, df_offer]:
        if 'id2' in df.columns:
            df['id2'] = df['id2'].astype(str)
        if 'id3' in df.columns:
            df['id3'] = df['id3'].astype(str)
    
    # Step 1: Build models from overlap data
    print("\n1. Finding overlapping customers...")
    overlap_customers = identify_overlap_customers(df_event, df_trans)
    
    print("\n2. Creating behavioral profiles on GPU...")
    profiles = create_customer_profiles_gpu(
        df_event, df_trans, 
        overlap_customers[:min(50000, len(overlap_customers))]
    )
    
    print("\n3. Creating behavioral clusters on GPU...")
    profiles_clustered, cluster_profiles, customer_clusters, scaler, kmeans = create_behavioral_clusters_gpu(
        profiles, n_clusters=n_clusters
    )
    
    print("\n4. Building industry propensity on GPU...")
    industry_propensity, default_ctr = build_industry_propensity_gpu(df_event, df_trans, df_offer)
    
    print("\n5. Creating behavioral archetypes on GPU...")
    archetype_profiles, archetype_ctr, arch_scaler, arch_kmeans = create_behavioral_archetypes_gpu(
        df_event, df_trans
    )
    
    # Step 2: Create features for train/val/test
    print("\n6. Creating features for training data...")
    train_features = create_features_gpu_batch(
        train_df, df_event, df_trans, df_offer,
        customer_clusters, cluster_profiles,
        industry_propensity, default_ctr,
        batch_size=batch_size
    )
    
    print("\n7. Creating features for validation data...")
    val_features = create_features_gpu_batch(
        val_df, df_event, df_trans, df_offer,
        customer_clusters, cluster_profiles,
        industry_propensity, default_ctr,
        batch_size=batch_size
    )
    
    print("\n8. Creating features for test data...")
    test_features = create_features_gpu_batch(
        test_df, df_event, df_trans, df_offer,
        customer_clusters, cluster_profiles,
        industry_propensity, default_ctr,
        batch_size=batch_size
    )
    
    # Merge features
    train_df_enhanced = pd.concat([train_df.reset_index(drop=True), train_features], axis=1)
    val_df_enhanced = pd.concat([val_df.reset_index(drop=True), val_features], axis=1)
    test_df_enhanced = pd.concat([test_df.reset_index(drop=True), test_features], axis=1)
    
    # List of new features
    new_features = [
        'temporal_days_to_nearest_holiday', 'temporal_nearest_holiday_idx', 'temporal_is_holiday_week',
        'temporal_holiday_type', 'behavioral_cluster', 'behavioral_cluster_ctr',
        'prob_industry_propensity', 'customer_avg_trans', 'customer_trans_count', 'customer_trans_cv'
    ]
    
    print("\n=== GPU Feature Engineering Complete ===")
    print(f"Training shape: {train_df_enhanced.shape}")
    print(f"Validation shape: {val_df_enhanced.shape}")
    print(f"Test shape: {test_df_enhanced.shape}")
    print("Feature engineering completed. Added GPU features:")
    print(new_features)
    
    return train_df_enhanced, val_df_enhanced, test_df_enhanced

# ==========================================
# QUICK EXAMPLE FUNCTION
# ==========================================

def quick_gpu_example(train_df, val_df, test_df, df_event, df_trans, df_offer, sample_size=10000):
    """Quick example with GPU acceleration"""
    try:
        print("Running GPU-accelerated feature engineering...")
        
        # Run pipeline
        train_gpu, val_gpu, test_gpu = add_gpu_features(
            train_df, val_df, test_df, df_event, df_trans, df_offer,
            sample_size=sample_size,
            batch_size=50000
        )
        
        print("\nSample of created features:")
        gpu_features = [col for col in train_gpu.columns 
                       if col.startswith(('temporal_', 'behavioral_', 'prob_', 'customer_'))]
        print(f"Created {len(gpu_features)} new features")
        print(gpu_features[:10])
        
        return train_gpu, val_gpu, test_gpu
        
    except Exception as e:
        print(f"Error: {e}")
        import traceback
        traceback.print_exc()
        return None, None, None

In [ ]:
train_df, val_df, test_df = quick_gpu_example(train_df, val_df, test_df, df_event, df_trans, df_offer, sample_size=False)
del df_event, df_trans, df_offer
gc.collect()

# Clear GPU memory if available
try:
    import cupy as cp
    cp.get_default_memory_pool().free_all_blocks()
    cp.get_default_pinned_memory_pool().free_all_blocks()
except:
    pass

print(f"Memory freed after GPU processing")

In [ ]:
# def add_bayesian_features(train_df: pd.DataFrame, val_df: pd.DataFrame, test_df: pd.DataFrame) -> tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
#     """
#     Add Bayesian-smoothed features to train, validation, and test DataFrames based on customer (id2) and offer (id3) metrics.
#     Avoids using the target variable 'y' to prevent target leakage.

#     Parameters:
#     - train_df: Training DataFrame with 'id2', 'id3', 'f132', 'best_day_ctr', 'overall_ctr'
#     - val_df: Validation DataFrame with 'id2', 'id3', 'f132', 'best_day_ctr', 'overall_ctr'
#     - test_df: Test DataFrame with 'id2', 'id3', 'f132', 'best_day_ctr', 'overall_ctr'

#     Returns:
#     - Tuple of (train_df, val_df, test_df) with Bayesian features added
#     """
#     # Create copies to avoid modifying original DataFrames
#     train_df = train_df.copy()
#     val_df = val_df.copy()
#     test_df = test_df.copy()

#     # Check for required columns
#     required_cols = ['id2', 'id3', 'f132', 'best_day_ctr', 'overall_ctr']
#     # Training and validation require 'y', but test may not
#     for df, name in [(train_df, 'train_df'), (val_df, 'val_df')]:
#         missing_cols = [col for col in required_cols + ['y'] if col not in df.columns]
#         if missing_cols:
#             raise KeyError(f"Missing columns in {name}: {missing_cols}")
#     # Test DataFrame may not have 'y'
#     missing_cols_test = [col for col in required_cols if col not in test_df.columns]
#     if missing_cols_test:
#         raise KeyError(f"Missing columns in test_df: {missing_cols_test}")

#     # Bayesian smoothing function
#     def bayesian_smoothing(group, value_col, prior_mean, prior_weight=10):
#         """
#         Apply Bayesian smoothing to a value column.
#         - prior_mean: Global mean of the value column
#         - prior_weight: Weight of the prior (controls smoothing strength)
#         """
#         counts = group.count()
#         means = group.mean()
#         smoothed = (means * counts + prior_mean * prior_weight) / (counts + prior_weight)
#         return smoothed

#     # 1. Bayesian-Smoothed Features for Customer (id2)
#     print("Adding Bayesian-smoothed features for customers (id2)...")
#     # Compute global means from training data only to avoid leakage
#     global_f132_mean = train_df['f132'].mean()
#     global_best_day_ctr_mean = train_df['best_day_ctr'].mean()
#     global_overall_ctr_mean = train_df['overall_ctr'].mean()

#     for df, name in [(train_df, 'train_df'), (val_df, 'val_df'), (test_df, 'test_df')]:
#         # Bayesian-smoothed f132 per customer
#         smoothed_f132 = df.groupby('id2')['f132'].apply(lambda x: bayesian_smoothing(x, 'f132', global_f132_mean))
#         df['bayes_f132_per_id2'] = df['id2'].map(smoothed_f132)

#         # Bayesian-smoothed best_day_ctr per customer
#         smoothed_best_day_ctr = df.groupby('id2')['best_day_ctr'].apply(lambda x: bayesian_smoothing(x, 'best_day_ctr', global_best_day_ctr_mean))
#         df['bayes_best_day_ctr_per_id2'] = df['id2'].map(smoothed_best_day_ctr)

#         # Bayesian-smoothed overall_ctr per customer
#         smoothed_overall_ctr = df.groupby('id2')['overall_ctr'].apply(lambda x: bayesian_smoothing(x, 'overall_ctr', global_overall_ctr_mean))
#         df['bayes_overall_ctr_per_id2'] = df['id2'].map(smoothed_overall_ctr)

#     # 2. Bayesian-Smoothed Features for Offer (id3)
#     print("Adding Bayesian-smoothed features for offers (id3)...")
#     for df, name in [(train_df, 'train_df'), (val_df, 'val_df'), (test_df, 'test_df')]:
#         # Bayesian-smoothed f132 per offer
#         smoothed_f132_offer = df.groupby('id3')['f132'].apply(lambda x: bayesian_smoothing(x, 'f132', global_f132_mean))
#         df['bayes_f132_per_id3'] = df['id3'].map(smoothed_f132_offer)

#         # Bayesian-smoothed best_day_ctr per offer
#         smoothed_best_day_ctr_offer = df.groupby('id3')['best_day_ctr'].apply(lambda x: bayesian_smoothing(x, 'best_day_ctr', global_best_day_ctr_mean))
#         df['bayes_best_day_ctr_per_id3'] = df['id3'].map(smoothed_best_day_ctr_offer)

#         # Bayesian-smoothed overall_ctr per offer
#         smoothed_overall_ctr_offer = df.groupby('id3')['overall_ctr'].apply(lambda x: bayesian_smoothing(x, 'overall_ctr', global_overall_ctr_mean))
#         df['bayes_overall_ctr_per_id3'] = df['id3'].map(smoothed_overall_ctr_offer)

#     # List of new features
#     new_features = [
#         'bayes_f132_per_id2', 'bayes_best_day_ctr_per_id2', 'bayes_overall_ctr_per_id2',
#         'bayes_f132_per_id3', 'bayes_best_day_ctr_per_id3', 'bayes_overall_ctr_per_id3'
#     ]
#     print("Bayesian feature engineering completed. Added features:")
#     print(new_features)

#     return train_df, val_df, test_df

In [ ]:
gc.collect()

In [ ]:
train_df, val_df, test_df =  add_new_features(train_df, val_df, test_df)
gc.collect()

In [ ]:
gc.collect()

In [ ]:
def add_bayesian_features(train_df, val_df, test_df):
    """Add Bayesian smoothed CTR features with hierarchical priors"""
    
    # Global priors from training data
    global_ctr = train_df['y'].mean()
    alpha, beta = 1, 1/global_ctr - 1  # Beta prior parameters
    
    # 1. Customer-level Bayesian CTR
    for df in [train_df, val_df, test_df]:
        # Get customer click/impression counts from features
        customer_stats = df.groupby('id2').agg({
            'f147': 'sum',  # clicks in last 30 days
            'f146': 'sum'   # impressions in last 14 days
        }).reset_index()
        
        # Bayesian smoothing
        customer_stats['bayesian_customer_ctr'] = (
            (customer_stats['f147'] + alpha) / 
            (customer_stats['f146'] + alpha + beta)
        )
        
        df = df.merge(customer_stats[['id2', 'bayesian_customer_ctr']], on='id2', how='left')
    
    # 2. Offer-level Bayesian CTR
    offer_stats = train_df.groupby('id3')['y'].agg(['sum', 'count']).reset_index()
    offer_stats['bayesian_offer_ctr'] = (
        (offer_stats['sum'] + alpha) / 
        (offer_stats['count'] + alpha + beta)
    )
    
    # 3. Customer-Category Bayesian CTR
    # Using f123-f145 (category-specific clicks)
    category_features = []
    for i in range(123, 146):
        if f'f{i}' in train_df.columns:
            category_features.append(f'f{i}')
    
    # 4. Time-decay weighted Bayesian CTR
    df['time_weighted_bayesian_ctr'] = (
        (df['f29'] + alpha) /  # decaying clicks
        (df['f28'] + alpha + beta)  # decaying impressions
    )
    
    return train_df, val_df, test_df
train_df, val_df, test_df = add_bayesian_features(train_df, val_df, test_df)

In [ ]:
import numpy as np
import pandas as pd
import cudf
import cupy as cp
from cuml.cluster import KMeans as cuKMeans
from cuml.preprocessing import StandardScaler as cuStandardScaler

def add_interest_segmentation_features_gpu(train_df, val_df, test_df):
    """Memory-efficient GPU feature engineering for interest segmentation"""

    interest_cols = [f'f{i}' for i in range(1, 13)]

    def add_features(df):
        df_gpu = cudf.DataFrame(df[interest_cols])  # Only f1-f12 to GPU
        arr = df_gpu.to_cupy()

        # 1. Entropy (vectorized)
        row_sums = cp.sum(arr, axis=1).reshape(-1, 1)
        p = cp.where(row_sums != 0, arr / row_sums, 0.0)
        entropy = -cp.sum(cp.where(p != 0, p * cp.log(p + 1e-10), 0.0), axis=1)

        # 2. Dominant interest
        dominant_interest = cp.argmax(arr, axis=1)
        dominant_interest_score = cp.max(arr, axis=1)

        # 3. Gini coefficient (vectorized)
        x_sorted = cp.sort(arr, axis=1)
        n = arr.shape[1]
        index = cp.arange(1, n + 1).reshape(1, -1)
        gini = (2 * cp.sum(index * x_sorted, axis=1)) / (n * cp.sum(arr, axis=1) + 1e-10) - (n + 1) / n

        # 4. Multi-interest indicator (> mean of the row)
        row_means = cp.mean(arr, axis=1).reshape(-1, 1)
        multi_interest_count = cp.sum(arr > row_means, axis=1)

        # Save features back to CPU dataframe
        df['interest_entropy'] = cp.asnumpy(entropy)
        df['dominant_interest'] = cp.asnumpy(dominant_interest)
        df['dominant_interest_score'] = cp.asnumpy(dominant_interest_score)
        df['interest_concentration'] = cp.asnumpy(gini)
        df['multi_interest_count'] = cp.asnumpy(multi_interest_count)

        return df

    # Add features to each dataframe
    train_df = add_features(train_df)
    val_df = add_features(val_df)
    test_df = add_features(test_df)

    # GPU clustering (KMeans)
    scaler = cuStandardScaler()
    train_gpu = cudf.DataFrame(train_df[interest_cols])
    train_scaled = scaler.fit_transform(train_gpu)

    kmeans = cuKMeans(n_clusters=20, random_state=42)
    train_df['interest_cluster'] = cp.asnumpy(kmeans.fit_predict(train_scaled))

    for df in [val_df, test_df]:
        df_gpu = cudf.DataFrame(df[interest_cols])
        df_scaled = scaler.transform(df_gpu)
        df['interest_cluster'] = cp.asnumpy(kmeans.predict(df_scaled))

    return train_df, val_df, test_df

train_df, val_df, test_df = add_interest_segmentation_features_gpu(train_df, val_df, test_df)

In [ ]:
def add_spend_velocity_features(train_df, val_df, test_df):
    """Create spend-based velocity and pattern features"""
    
    # Spend columns: f39 (Lifestyle), f40 (Electronics), f41 (Restaurant)
    spend_cols = ['f39', 'f40', 'f41']
    
    for df in [train_df, val_df, test_df]:
        # 1. Total spend across categories
        df['total_spend_3m'] = df[spend_cols].sum(axis=1)
        
        # 2. Spend diversity (which categories)
        df['spend_categories_active'] = (df[spend_cols] > 0).sum(axis=1)
        
        # 3. Spend concentration
        df['spend_concentration'] = df[spend_cols].max(axis=1) / (df['total_spend_3m'] + 1e-10)
        
        # 4. Category spend ratios
        for col in spend_cols:
            df[f'{col}_ratio'] = df[col] / (df['total_spend_3m'] + 1e-10)
        
        # 5. Spend vs. click alignment
        df['restaurant_spend_click_ratio'] = df['f41'] / (df['f125'] + 1)  # f125: restaurant clicks
        df['electronics_spend_impression_ratio'] = df['f40'] / (df['f366'] + 1)
        
        # 6. Big spender indicator
        df['is_big_spender'] = (df['total_spend_3m'] > df['total_spend_3m'].quantile(0.9)).astype(int)
        
        # 7. Spend velocity (if we had historical data)
        # This would compare 3-month spend to previous periods
        
    return train_df, val_df, test_df

train_df, val_df, test_df = add_spend_velocity_features(train_df, val_df, test_df)


In [ ]:
import numpy as np

def add_page_engagement_features(train_df, val_df, test_df):
    """Create features from page visit patterns, using only available columns"""

    # Define the expected columns
    page_visit_cols = [f'f{i}' for i in range(14, 22)]
    time_30d_cols = [f'f{i}' for i in range(59, 68)]
    time_180d_cols = [f'f{i}' for i in range(68, 77)]

    for df in [train_df, val_df, test_df]:
        # 1. Page visit count (only use available columns)
        existing_page_cols = [col for col in page_visit_cols if col in df.columns]
        if existing_page_cols:
            df['pages_visited_count'] = df[existing_page_cols].sum(axis=1)
        else:
            df['pages_visited_count'] = 0

        # 2. Engagement intensity
        df['total_time_30d'] = df['f59'] if 'f59' in df.columns else 0
        df['total_time_180d'] = df['f68'] if 'f68' in df.columns else 1e-10  # Avoid division by zero

        # 3. Recency ratio
        df['engagement_recency_ratio'] = df['total_time_30d'] / (df['total_time_180d'] + 1e-10)

        # 4. Page-specific engagement rates
        df['redemption_engagement'] = (df['f14'] * df['f61']) if 'f14' in df.columns and 'f61' in df.columns else 0
        df['travel_engagement'] = (df['f18'] * df['f65']) if 'f18' in df.columns and 'f65' in df.columns else 0

        # 5. Engagement consistency (CV of page times)
        existing_time_30d_cols = [col for col in time_30d_cols if col in df.columns]
        if existing_time_30d_cols:
            df['page_time_cv'] = df[existing_time_30d_cols].std(axis=1) / (df[existing_time_30d_cols].mean(axis=1) + 1e-10)
        else:
            df['page_time_cv'] = 0

        # 6. Focus score
        if existing_time_30d_cols:
            page_times = df[existing_time_30d_cols].values
            df['page_focus_score'] = np.max(page_times, axis=1) / (np.sum(page_times, axis=1) + 1e-10)
        else:
            df['page_focus_score'] = 0

        # 7. Engagement trend
        df['engagement_trend'] = (df['f77'] - 0.5) * 2 if 'f77' in df.columns else 0

    return train_df, val_df, test_df

train_df, val_df, test_df = add_page_engagement_features(train_df, val_df, test_df)

In [ ]:
def add_advanced_temporal_features(train_df, val_df, test_df):
    """Create sophisticated temporal features"""
    
    for df in [train_df, val_df, test_df]:
        # 1. Click velocity changes
        df['click_acceleration'] = (df['f201'] - df['f147']) / 14  # 14-day vs 30-day
        
        # 2. Impression fatigue
        df['impression_fatigue_score'] = df['f203'] / (df['f313'] + 1e-10)  # 30d vs 14d CTR
        
        # 3. Temporal consistency
        # Using various time window features
        click_windows = {
            '7d': 'f209',   # 7-day clicks
            '14d': 'f201',  # 14-day clicks
            '30d': 'f147',  # 30-day clicks
            '60d': 'f123',  # 60-day clicks (dining category as proxy)
        }
        
        if all(col in df.columns for col in click_windows.values()):
            click_values = [df[col] for col in click_windows.values()]
            df['click_temporal_std'] = np.std(click_values, axis=0)
            df['click_temporal_cv'] = df['click_temporal_std'] / (np.mean(click_values, axis=0) + 1e-10)
        
        # 4. Day-of-week patterns (from existing datetime features)
        if 'day_of_week' in df.columns:
            # Assuming we can aggregate historical data
            df['is_weekend_person'] = df['day_of_week'].isin([5, 6]).astype(int)
        
        # 5. Response time features
        # Time between impression and click (would need event-level data)
        
    return train_df, val_df, test_df
train_df, val_df, test_df = add_advanced_temporal_features(train_df, val_df, test_df)

In [ ]:
gc.collect()

In [ ]:
def add_statistical_features(train_df, val_df, test_df):
    """Add statistical distribution features"""
    
    for df in [train_df, val_df, test_df]:
        # 1. Z-scores for key metrics
        for col in ['f132', 'f366', 'overall_ctr']:
            mean = train_df[col].mean()
            std = train_df[col].std()
            df[f'{col}_zscore'] = (df[col] - mean) / (std + 1e-10)
        
        # 2. Percentile ranks
        for col in ['f132', 'best_day_ctr', 'f366']:
            df[f'{col}_percentile'] = df[col].rank(pct=True)
        
        # 3. Outlier indicators
        df['is_ctr_outlier'] = (np.abs(df['overall_ctr_zscore']) > 3).astype(int)
        
        # 4. Relative performance metrics
        df['ctr_vs_median'] = df['overall_ctr'] / df['overall_ctr'].median()
        
        # 5. Distribution moments
        numeric_cols = ['f132', 'f366', 'best_day_ctr', 'worst_day_ctr']
        df['metric_skewness'] = df[numeric_cols].apply(lambda x: x.skew(), axis=1)
        df['metric_kurtosis'] = df[numeric_cols].apply(lambda x: x.kurtosis(), axis=1)
        
    return train_df, val_df, test_df
train_df, val_df, test_df = add_statistical_features(train_df, val_df, test_df)


In [ ]:
def add_cross_interactions(train_df, val_df, test_df):
    """Create meaningful cross-feature interactions"""
    
    for df in [train_df, val_df, test_df]:
        # 1. Interest × Spend interactions
        df['travel_interest_spend_alignment'] = df['f8'] * df['f65']  # Travel interest × travel page time
        
        # 2. CTR × Category interactions
        df['high_ctr_dining'] = df['overall_ctr'] * df['f125']  # CTR × dining clicks
        
        # 3. Temporal × Behavioral interactions
        df['weekend_high_value'] = df.get('is_weekend_person', 0) * df.get('is_big_spender', 0)
        
        # 4. Engagement × Performance
        df['engaged_high_ctr'] = df['f59'] * df['overall_ctr']  # Time spent × CTR
        
        # 5. Offer × Customer type interactions
        df['elite_premium_offer'] = df['f46'] * df.get('offer_value_score', 0)
        
        # 6. Polynomial features for top predictors
        top_features = ['f132', 'f366', 'best_day_ctr']
        for feat in top_features:
            df[f'{feat}_squared'] = df[feat] ** 2
            df[f'{feat}_log'] = np.log1p(df[feat])
        
        # 7. Ratio interactions
        df['impression_click_efficiency'] = df['f147'] / (df['f146'] + 1) * df['overall_ctr']
        
    return train_df, val_df, test_df
train_df, val_df, test_df = add_cross_interactions(train_df, val_df, test_df)


In [ ]:
def add_merchant_affinity_features(train_df, val_df, test_df):
    """Create merchant and offer-type affinity features"""
    
    # Offer type CTR features (f104-f117)
    offer_type_cols = {
        'rewards': 'f104',
        'benefit': 'f105',
        'acquisition': 'f106',
        'upgrade': 'f107',
        'retail': 'f108',
        'servicing': 'f111',
        'insurance': 'f112'
    }
    
    for df in [train_df, val_df, test_df]:
        # 1. Best performing offer type
        offer_ctrs = df[[col for col in offer_type_cols.values() if col in df.columns]]
        df['best_offer_type_ctr'] = offer_ctrs.max(axis=1)
        df['preferred_offer_type'] = offer_ctrs.idxmax(axis=1)
        
        # 2. Offer type diversity
        df['offer_type_breadth'] = (offer_ctrs > 0).sum(axis=1)
        
        # 3. Merchant vs non-merchant preference
        df['merchant_preference_ratio'] = df['f203'] / (df['f110'] + 1e-10)
        
        # 4. Category-specific momentum
        df['dining_momentum'] = df['f123'] / (df['f124'] + 1)  # Recent vs historical
        
    return train_df, val_df, test_df

train_df, val_df, test_df = add_merchant_affinity_features(train_df, val_df, test_df)


In [ ]:
def add_meta_features(train_df, val_df, test_df):
    """Create meta-features from other features, handling missing columns safely"""
    
    for df in [train_df, val_df, test_df]:
        # 1. Feature completeness (only use features that exist)
        important_features = ['f132', 'f366', 'best_day_ctr', 'f209', 'f313']
        available_important = [col for col in important_features if col in df.columns]
        
        if available_important:
            df['feature_completeness'] = df[available_important].notna().sum(axis=1) / len(important_features)
        else:
            df['feature_completeness'] = 0
        
        # 2. Data density score
        f146 = df['f146'] if 'f146' in df.columns else 1
        transaction_count = df['transaction_count_last_12h'] if 'transaction_count_last_12h' in df.columns else 1

        df['data_density_score'] = (
            transaction_count *
            f146 *
            df['feature_completeness']
        ) ** (1/3)
        
        # 3. Anomaly score (sum of absolute z-scores)
        zscore_cols = [col for col in df.columns if col.endswith('_zscore')]
        if zscore_cols:
            df['anomaly_score'] = df[zscore_cols].abs().sum(axis=1)
        else:
            df['anomaly_score'] = 0
        
        # 4. Engagement complexity
        engagement_indicators = [f'f{i}' for i in range(14, 22)]
        available_engagement = [col for col in engagement_indicators if col in df.columns]

        if available_engagement:
            df['engagement_complexity'] = df[available_engagement].sum(axis=1)
        else:
            df['engagement_complexity'] = 0

    return train_df, val_df, test_df

train_df, val_df, test_df = add_meta_features(train_df, val_df, test_df)

In [ ]:
gc.collect()

In [ ]:
def add_hyper_temporal_features(train_df, val_df, test_df):
    """Create advanced temporal velocity and acceleration features"""
    
    for df in [train_df, val_df, test_df]:
        # 1. Multi-scale velocity ratios
        # Building on success of transaction_count_last_12h
        if 'transaction_count_last_12h' in df.columns:
            df['transaction_velocity_6h_vs_12h'] = (
                df.get('transaction_count_last_6h', 0) * 2 / 
                (df['transaction_count_last_12h'] + 1)
            )
            df['transaction_acceleration'] = (
                df['transaction_velocity_6h_vs_12h'] - 0.5
            ) * 2  # Normalized acceleration
        
        # 2. Click velocity harmonics (building on max/min/avg click velocity)
        if all(col in df.columns for col in ['max_click_velocity', 'min_click_velocity', 'avg_click_velocity']):
            df['click_velocity_range'] = df['max_click_velocity'] - df['min_click_velocity']
            df['click_velocity_stability'] = 1 / (df['click_velocity_range'] + 1)
            df['click_velocity_skew'] = (
                (df['max_click_velocity'] - df['avg_click_velocity']) /
                (df['avg_click_velocity'] - df['min_click_velocity'] + 1e-6)
            )
        
        # 3. Temporal feature derivatives
        if 'f209' in df.columns and 'f313' in df.columns:  # 7-day and 14-day CTR
            df['ctr_momentum_7_14'] = df['f209'] / (df['f313'] + 1e-6)
            df['ctr_jerk'] = df['ctr_momentum_7_14'] - 1  # Rate of change of momentum
        
        # 4. Complex temporal patterns
        if 'time_since_prev_txn_hours' in df.columns:
            df['transaction_regularity_score'] = 1 / (
                df.groupby('id2')['time_since_prev_txn_hours'].transform('std') + 1
            )
            df['is_bursty_user'] = (
                df['time_since_prev_txn_hours'] < 
                df['time_since_prev_txn_hours'].quantile(0.1)
            ).astype(int)
        
        # 5. Engagement intensity waves
        if 'engagement_intensity' in df.columns and 'f132' in df.columns:
            df['engagement_f132_resonance'] = (
                df['engagement_intensity'] * df['f132'] / 
                (df['engagement_intensity'] + df['f132'] + 1e-6)
            )  # Harmonic mean
            
        # 6. Performance floor dynamics
        if 'performance_floor' in df.columns and 'worst_day_ctr' in df.columns:
            df['floor_to_worst_ratio'] = df['performance_floor'] / (df['worst_day_ctr'] + 1e-6)
            df['performance_resilience'] = np.minimum(df['floor_to_worst_ratio'], 10)
        
    return train_df, val_df, test_df
train_df, val_df, test_df = add_hyper_temporal_features(train_df, val_df, test_df)

In [ ]:
gc.collect()

In [ ]:
def add_advanced_statistical_features(train_df, val_df, test_df):
    """Create sophisticated statistical features building on zscore/percentile success"""
    
    for df in [train_df, val_df, test_df]:
        # 1. Multi-metric statistical profiles
        stat_cols = ['f132', 'f366', 'overall_ctr', 'best_day_ctr']
        if all(col in df.columns for col in stat_cols):
            # Mahalanobis distance from "ideal" customer
            from scipy.spatial.distance import mahalanobis
            ideal_profile = df[stat_cols].quantile(0.9).values
            cov_matrix = df[stat_cols].cov().values
            cov_inv = np.linalg.pinv(cov_matrix)
            
            df['mahalanobis_distance'] = df[stat_cols].apply(
                lambda x: mahalanobis(x.values, ideal_profile, cov_inv), axis=1
            )
            
        # 2. Extreme value indicators
        if 'f132_zscore' in df.columns and 'overall_ctr_zscore' in df.columns:
            df['is_extreme_performer'] = (
                (np.abs(df['f132_zscore']) > 2) & 
                (np.abs(df['overall_ctr_zscore']) > 2)
            ).astype(int)
            
            df['zscore_manhattan'] = (
                np.abs(df['f132_zscore']) + 
                np.abs(df['overall_ctr_zscore'])
            )
            
        # 3. Volatility-adjusted metrics
        if 'daily_ctr_volatility' in df.columns:
            df['sharpe_ratio'] = df['overall_ctr'] / (df['daily_ctr_volatility'] + 1e-6)
            df['volatility_penalty'] = np.exp(-df['daily_ctr_volatility'])
            
        # 4. Distribution tail features
        if 'f366_percentile' in df.columns and 'f132_percentile' in df.columns:
            df['is_top_decile_both'] = (
                (df['f366_percentile'] > 0.9) & 
                (df['f132_percentile'] > 0.9)
            ).astype(int)
            
            df['percentile_product'] = df['f366_percentile'] * df['f132_percentile']
            df['percentile_gap'] = np.abs(df['f366_percentile'] - df['f132_percentile'])
            
        # 5. Higher-order moments interaction
        if 'metric_kurtosis' in df.columns:
            df['kurtosis_risk_score'] = np.clip(df['metric_kurtosis'] / 3, -1, 1)
            df['is_fat_tail'] = (df['metric_kurtosis'] > 3).astype(int)
        
    return train_df, val_df, test_df
train_df, val_df, test_df = add_advanced_statistical_features(train_df, val_df, test_df)

In [ ]:
gc.collect()

In [ ]:
def add_hyper_customer_features(train_df, val_df, test_df):
    """Create advanced customer-level features"""
    
    for df in [train_df, val_df, test_df]:
        # 1. Customer performance stability
        if 'max_best_day_ctr_per_id2' in df.columns and 'mean_f132_per_id2' in df.columns:
            df['customer_ctr_coefficient_variation'] = (
                df.groupby('id2')['best_day_ctr'].transform('std') /
                (df['mean_f132_per_id2'] + 1e-6)
            )
            
            df['customer_peak_to_average'] = (
                df['max_best_day_ctr_per_id2'] / 
                (df['mean_f132_per_id2'] + 1e-6)
            )
            
        # 2. Customer engagement patterns
        if 'customer_trans_count' in df.columns and 'customer_trans_cv' in df.columns:
            df['customer_consistency_score'] = (
                df['customer_trans_count'] / 
                (1 + df['customer_trans_cv'])
            )
            
            df['is_power_user'] = (
                (df['customer_trans_count'] > df['customer_trans_count'].quantile(0.8)) &
                (df['customer_trans_cv'] < df['customer_trans_cv'].quantile(0.2))
            ).astype(int)
            
        # 3. Customer market position
        if 'click_market_share' in df.columns and 'impression_market_share' in df.columns:
            df['market_efficiency_ratio'] = (
                df['click_market_share'] / 
                (df['impression_market_share'] + 1e-6)
            )
            
            df['market_dominance_score'] = (
                df['click_market_share'] * df['impression_market_share']
            ) ** 0.5
            
        # 4. Customer quality metrics
        if all(col in df.columns for col in ['quality_score', 'reliability_score', 'success_rate']):
            df['composite_quality_score'] = (
                df['quality_score'] * 0.4 +
                df['reliability_score'] * 0.3 +
                df['success_rate'] * 0.3
            )
            
            df['quality_consistency'] = 1 - df[
                ['quality_score', 'reliability_score', 'success_rate']
            ].std(axis=1)
            
    return train_df, val_df, test_df

train_df, val_df, test_df = add_hyper_customer_features(train_df, val_df, test_df)

In [ ]:
def add_nonlinear_combinations(train_df, val_df, test_df):
    """Create non-linear combinations of top features"""
    
    for df in [train_df, val_df, test_df]:
        # 1. Polynomial interactions of top features
        if 'f366_squared' in df.columns and 'f132' in df.columns:
            df['f366_f132_product'] = df['f366'] * df['f132']
            df['f366_f132_ratio_squared'] = (df['f366'] / (df['f132'] + 1)) ** 2
            
        # 2. Log-scale interactions
        if 'f366_log' in df.columns and 'best_day_ctr_squared' in df.columns:
            df['log_squared_interaction'] = df['f366_log'] * np.sqrt(df['best_day_ctr_squared'])
            
        # 3. Trigonometric combinations
        if 'f132' in df.columns and 'f366' in df.columns:
            # Map to [0, 2π] range
            f132_norm = df['f132'] / (df['f132'].max() + 1) * 2 * np.pi
            f366_norm = df['f366'] / (df['f366'].max() + 1) * 2 * np.pi
            
            df['f132_f366_phase'] = np.sin(f132_norm) * np.cos(f366_norm)
            df['f132_f366_magnitude'] = np.sqrt(
                np.sin(f132_norm)**2 + np.cos(f366_norm)**2
            )
            
        # 4. Exponential decay combinations
        if 'impression_fatigue_score' in df.columns and 'engagement_intensity' in df.columns:
            df['fatigue_adjusted_engagement'] = (
                df['engagement_intensity'] * 
                np.exp(-df['impression_fatigue_score'])
            )
            
        # 5. Threshold-based transformations
        if 'ctr_vs_median' in df.columns:
            df['ctr_sigmoid_transform'] = 1 / (1 + np.exp(-2 * (df['ctr_vs_median'] - 1)))
            df['ctr_relu_transform'] = np.maximum(0, df['ctr_vs_median'] - 1)
            
    return train_df, val_df, test_df

train_df, val_df, test_df = add_nonlinear_combinations(train_df, val_df, test_df)

In [ ]:
def add_market_intelligence_features(train_df, val_df, test_df):
    """Create sophisticated market comparison features"""
    
    for df in [train_df, val_df, test_df]:
        # 1. Market position dynamics
        if 'above_market_avg' in df.columns:
            df['market_outperformance_degree'] = np.log1p(
                np.maximum(0, df['above_market_avg'])
            )
            
            df['consistent_outperformer'] = (
                df.groupby('id2')['above_market_avg'].transform(
                    lambda x: (x > 0).mean()
                )
            )
            
        # 2. Competitive advantage metrics
        if 'customer_penetration' in df.columns and 'exposure_efficiency' in df.columns:
            df['competitive_advantage_score'] = (
                df['customer_penetration'] * df['exposure_efficiency']
            )
            
            df['penetration_efficiency_ratio'] = (
                df['customer_penetration'] / 
                (1 - df['exposure_efficiency'] + 1e-6)
            )
            
        # 3. Market share velocity
        if 'click_market_share' in df.columns:
            df['market_share_momentum'] = df.groupby('id2')['click_market_share'].transform(
                lambda x: x.diff().fillna(0)
            )
            
            df['market_share_acceleration'] = df.groupby('id2')['market_share_momentum'].transform(
                lambda x: x.diff().fillna(0)
            )
            
    return train_df, val_df, test_df

train_df, val_df, test_df = add_market_intelligence_features(train_df, val_df, test_df)

In [ ]:
gc.collect()

In [ ]:
import cudf
import cupy as cp

def add_deep_temporal_patterns_gpu(train_df, val_df, test_df):
    """GPU-accelerated deep temporal patterns"""

    time_windows_dict = {'f29': 30, 'f123': 60, 'f147': 30, 'f201': 14, 'f209': 7}

    results = {}

    for name, df in zip(['train', 'val', 'test'], [train_df, val_df, test_df]):
        df = cudf.from_pandas(df)

        # 1. Multi-resolution temporal features
        available_cols = [col for col in time_windows_dict.keys() if col in df.columns]
        
        if len(available_cols) >= 3:
            temporal_values = df[available_cols].to_cupy()
            df['temporal_consistency'] = 1 / (cp.std(temporal_values, axis=1) + 1)

            if len(available_cols) >= 4:
                X = cp.array([time_windows_dict[col] for col in available_cols])
                X_mean = cp.mean(X)
                
                y = temporal_values
                y_mean = cp.mean(y, axis=1)

                numerator = cp.sum((X - X_mean) * (y - y_mean[:, None]), axis=1)
                denominator = cp.sum((X - X_mean) ** 2)

                slopes = numerator / (denominator + 1e-10)
                df['temporal_trend_slope'] = slopes

        # 2. Cyclical pattern detection
        if 'dining_momentum' in df.columns and 'f125' in df.columns:
            df['dining_cycle_strength'] = (
                df['dining_momentum'] * cp.sin(df['f125'].to_cupy() / (df['f125'].max() + 1e-10) * 2 * cp.pi)
            )
        
        # 3. Offer response patterns
        offer_cols = ['f343', 'f344', 'f345']
        if all(col in df.columns for col in offer_cols):
            offers = df[offer_cols].to_cupy()
            df['offer_response_variance'] = cp.var(offers, axis=1)

            row_sums = cp.sum(offers, axis=1)
            probs = offers / (row_sums[:, None] + 1e-10)
            entropy = -cp.sum(probs * cp.log(probs + 1e-10), axis=1)
            df['offer_response_entropy'] = entropy

        results[name] = df

    # Return in correct order and convert to pandas
    return (
        results['train'].to_pandas(),
        results['val'].to_pandas(),
        results['test'].to_pandas()
    )


train_df, val_df, test_df =  add_deep_temporal_patterns_gpu(train_df, val_df, test_df)

In [ ]:
def add_feature_interaction_networks(train_df, val_df, test_df):
    """Create complex feature interaction networks"""
    
    for df in [train_df, val_df, test_df]:
        # 1. Performance network features
        perf_features = ['success_rate', 'quality_score', 'reliability_score', 'performance_consistency']
        available_perf = [f for f in perf_features if f in df.columns]
        
        if len(available_perf) >= 3:
            # Create interaction network
            for i, feat1 in enumerate(available_perf):
                for feat2 in available_perf[i+1:]:
                    df[f'{feat1}_x_{feat2}'] = df[feat1] * df[feat2]
                    df[f'{feat1}_div_{feat2}'] = df[feat1] / (df[feat2] + 1e-6)
                    
        # 2. Click-CTR interaction network  
        click_features = ['total_clicks', 'immediate_click_rate', 'quick_click_rate']
        ctr_features = ['f313', 'overall_ctr', 'worst_day_ctr']
        
        for click_feat in click_features:
            for ctr_feat in ctr_features:
                if click_feat in df.columns and ctr_feat in df.columns:
                    df[f'{click_feat}_weighted_by_{ctr_feat}'] = (
                        df[click_feat] * df[ctr_feat]
                    )
                    
        # 3. Spend-engagement network
        if all(col in df.columns for col in ['f39_ratio', 'f40_ratio', 'f41_ratio']):
            df['spend_diversity_index'] = 1 - df[['f39_ratio', 'f40_ratio', 'f41_ratio']].max(axis=1)
            df['spend_focus_category'] = df[['f39_ratio', 'f40_ratio', 'f41_ratio']].idxmax(axis=1)
            
    return train_df, val_df, test_df

train_df, val_df, test_df = add_feature_interaction_networks(train_df, val_df, test_df)

In [ ]:
def add_meta_features2(train_df, val_df, test_df):
    """Create meta-features from existing feature patterns"""
    
    for df in [train_df, val_df, test_df]:
        # 1. Feature quality indicators
        if 'feature_completeness' in df.columns:
            df['high_quality_profile'] = (df['feature_completeness'] > 0.8).astype(int)
            df['feature_richness'] = df.notna().sum(axis=1) / len(df.columns)
            
        # 2. Consistency across feature families
        feature_families = {
            'velocity': ['max_click_velocity', 'min_click_velocity', 'avg_click_velocity'],
            'ctr': ['overall_ctr', 'worst_day_ctr', 'best_day_ctr'],
            'market': ['click_market_share', 'impression_market_share', 'above_market_avg']
        }
        
        for family_name, family_features in feature_families.items():
            available = [f for f in family_features if f in df.columns]
            if len(available) >= 2:
                df[f'{family_name}_consistency'] = 1 / (df[available].std(axis=1) + 1)
                df[f'{family_name}_range'] = df[available].max(axis=1) - df[available].min(axis=1)
                
        # 3. Anomaly composite scores
        anomaly_indicators = ['is_ctr_outlier', 'is_extreme_performer', 'is_fat_tail']
        available_anomaly = [f for f in anomaly_indicators if f in df.columns]
        if available_anomaly:
            df['anomaly_score'] = df[available_anomaly].sum(axis=1)
            df['is_super_anomaly'] = (df['anomaly_score'] >= 2).astype(int)
            
        # 4. Feature interaction strength
        if 'f132_zscore' in df.columns and 'f366_percentile' in df.columns:
            df['feature_discord'] = np.abs(
                df['f132_zscore'] - (df['f366_percentile'] - 0.5) * 4
            )
            
    return train_df, val_df, test_df


train_df, val_df, test_df = add_meta_features2(train_df, val_df, test_df)

In [ ]:
gc.collect()

In [ ]:
def add_offer_customer_matching(train_df, val_df, test_df):
    """Create sophisticated offer-customer matching features"""
    
    for df in [train_df, val_df, test_df]:
        # 1. Offer type affinity scores
        if 'preferred_offer_type' in df.columns and 'f111' in df.columns:
            # Create binary indicators for each offer type preference
            offer_types = df['preferred_offer_type'].unique()
            for otype in offer_types:
                df[f'prefers_{otype}'] = (df['preferred_offer_type'] == otype).astype(int)
                
        # 2. Temporal offer matching
        if 'offer_start_hour' in df.columns and 'interaction_hour' in df.columns:
            df['hour_alignment_score'] = 1 - np.abs(
                df['offer_start_hour'] - df['interaction_hour']
            ) / 12  # Normalized to [0,1]
            
            df['is_prime_time_match'] = (
                (df['offer_start_hour'].between(9, 11) | df['offer_start_hour'].between(19, 21)) &
                (df['interaction_hour'].between(9, 11) | df['interaction_hour'].between(19, 21))
            ).astype(int)
            
        # 3. Offer freshness impact
        if 'offer_freshness_category' in df.columns:
            freshness_map = {'Fresh': 3, 'Moderate': 2, 'Stale': 1}
            df['freshness_score'] = df['offer_freshness_category'].map(freshness_map).fillna(0)
            
            if 'engagement_intensity' in df.columns:
                df['freshness_engagement_product'] = (
                    df['freshness_score'] * df['engagement_intensity']
                )
                
        # 4. Discount optimization features
        if all(col in df.columns for col in ['discount_above_median_behavioral', 'discount_scarcity']):
            df['discount_attractiveness'] = (
                df['discount_above_median_behavioral'] * df['discount_scarcity']
            )
            
            if 'is_big_spender' in df.columns:
                df['premium_discount_match'] = (
                    df['is_big_spender'] * df['discount_above_median_behavioral']
                )
                
    return train_df, val_df, test_df

train_df, val_df, test_df = add_offer_customer_matching(train_df, val_df, test_df)

In [ ]:
def select_final_features(train_df, model, top_150_features, new_features):
    """Select final features combining top 150 + best new features"""
    
    # Get importance for all features
    feature_cols = [col for col in train_df.columns if col not in ['y', 'id1', 'id2', 'id3', 'id4', 'id5']]
    importance = model.feature_importance(importance_type='gain')
    feature_importance = dict(zip(feature_cols, importance))
    
    # Sort new features by importance
    new_feature_importance = {
        feat: feature_importance.get(feat, 0) 
        for feat in new_features if feat in feature_importance
    }
    
    # Select top 50 new features
    top_new_features = sorted(
        new_feature_importance.items(), 
        key=lambda x: x[1], 
        reverse=True
    )[:50]
    
    # Combine with original top 150
    final_features = list(set(top_150_features + [f[0] for f in top_new_features]))
    
    print(f"Final feature set: {len(final_features)} features")
    print(f"Top 10 new features added:")
    for feat, imp in top_new_features[:10]:
        print(f"  {feat}: {imp:.2f}")
        
    return final_features

In [ ]:
gc.collect()

In [ ]:
print(train_df.shape, 
val_df.shape,
test_df.shape)

In [ ]:
gc.collect()

In [ ]:
def prepare_data_for_lgb(train_df: pd.DataFrame, val_df: pd.DataFrame) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Prepare data for LightGBM by converting object columns to numeric
    """
    train_df_copy = train_df.copy()
    val_df_copy = val_df.copy()
    
    # Get feature columns (exclude target 'y')
    feature_cols = [col for col in train_df_copy.columns if col not in ['id1', 'id3', 'id4', 'id5', 'id6', 'id7', 'id8','id9', 'id10', 'id11', 'id12', 'id13','id2','y' 'pred']]
    
    # Find object columns
    object_cols = []
    for col in feature_cols:
        if train_df_copy[col].dtype == 'object':
            object_cols.append(col)
    
    print(f"Found {len(object_cols)} object columns to convert:")
    print(object_cols)
    
    # Convert object columns to numeric
    for col in object_cols:
        print(f"Converting column: {col}")
        
        # Combine train and val data for consistent encoding
        combined_data = pd.concat([train_df_copy[col], val_df_copy[col]], axis=0)
        
        # Try to convert to numeric first (in case they're numeric strings)
        try:
            combined_numeric = pd.to_numeric(combined_data, errors='coerce')
            if combined_numeric.notna().sum() > len(combined_data) * 0.8:  # If 80%+ can be converted
                train_df_copy[col] = pd.to_numeric(train_df_copy[col], errors='coerce')
                val_df_copy[col] = pd.to_numeric(val_df_copy[col], errors='coerce')
                print(f"  -> Converted to numeric")
                continue
        except:
            pass
        
        # If not numeric, use label encoding
        from sklearn.preprocessing import LabelEncoder
        le = LabelEncoder()
        
        # Fit on combined data to ensure consistent encoding
        combined_data_filled = combined_data.fillna('missing')
        le.fit(combined_data_filled)
        
        # Transform train and val data
        train_filled = train_df_copy[col].fillna('missing')
        val_filled = val_df_copy[col].fillna('missing')
        
        train_df_copy[col] = le.transform(train_filled)
        val_df_copy[col] = le.transform(val_filled)
        
        print(f"  -> Label encoded ({len(le.classes_)} unique values)")
    
    # Convert any remaining object columns to numeric
    for col in feature_cols:
        if train_df_copy[col].dtype == 'object':
            train_df_copy[col] = pd.to_numeric(train_df_copy[col], errors='coerce').fillna(0)
            val_df_copy[col] = pd.to_numeric(val_df_copy[col], errors='coerce').fillna(0)
    
    # Check final dtypes
    print("\nFinal data types:")
    feature_dtypes = train_df_copy[feature_cols].dtypes
    object_remaining = feature_dtypes[feature_dtypes == 'object']
    if len(object_remaining) > 0:
        print(f"WARNING: Still have object columns: {object_remaining.index.tolist()}")
    else:
        print("✓ All feature columns are numeric")
    
    return train_df_copy, val_df_copy

train_df, val_df = prepare_data_for_lgb(train_df, val_df)
gc.collect()


In [ ]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.feature_selection import mutual_info_classif
from sklearn.impute import SimpleImputer
import shap
import gc

# Step 1: LightGBM GPU Feature Importance
def get_lgb_gpu_importance(train_df, feature_cols):
    params = {
        'objective': 'binary',
        'boosting_type': 'gbdt',
        'metric': 'auc',
        'learning_rate': 0.05,
        'num_leaves': 31,
        'max_depth': -1,
        'feature_fraction': 0.8,
        'bagging_fraction': 0.8,
        'bagging_freq': 5,
        'min_data_in_leaf': 100,
        'verbosity': -1,
        'device': 'gpu',
        'gpu_platform_id': 0,
        'gpu_device_id': 0,
        'seed': 42
    }

    dtrain = lgb.Dataset(train_df[feature_cols].astype(np.float32), label=train_df['y'])
    
    model = lgb.train(params, dtrain, num_boost_round=300)
    
    imp_df = pd.DataFrame({
        'feature': feature_cols,
        'importance': model.feature_importance(importance_type='gain')
    }).sort_values(by='importance', ascending=False)
    
    top_features = imp_df.head(300)['feature'].tolist()
    
    del dtrain, model
    gc.collect()
    
    return top_features, imp_df

import cudf
import cupy as cp
import numpy as np
import pandas as pd
import gc

def cupy_rankdata(array):
    """Cupy's version of rankdata with average ranking for ties and NaN-safe."""
    n = array.size
    ranks = cp.zeros(n, dtype=cp.float32)

    nan_mask = cp.isnan(array)
    not_nan = ~nan_mask

    valid_vals = array[not_nan]
    sorter = cp.argsort(valid_vals)
    inv = cp.empty_like(sorter)
    inv[sorter] = cp.arange(len(valid_vals))

    sorted_vals = valid_vals[sorter]
    obs = cp.concatenate((cp.array([True], dtype=cp.bool_), (sorted_vals[1:] != sorted_vals[:-1])))

    dense_rank = cp.cumsum(obs).astype(cp.float32)

    counts = cp.bincount(dense_rank.astype(cp.int32))
    avg_ranks = cp.cumsum(counts) / counts
    ranks_non_nan = avg_ranks[dense_rank.astype(cp.int32) - 1]

    ranks[not_nan] = ranks_non_nan
    ranks[nan_mask] = cp.nan

    return ranks

def gpu_mutual_info_proxy(train_df, feature_cols):
    """
    GPU-based mutual information proxy using Spearman rank correlation.
    **NOW FULLY NULL-SAFE**
    """

    print("Step 2: GPU Mutual Info Approximation with Null Handling...")

    # Convert to cudf
    X_cudf = cudf.DataFrame(train_df[feature_cols].astype(np.float32))
    y_cudf = cudf.Series(train_df['y'].astype(np.int8))

    # Fill NaNs before .to_cupy() to avoid cudf error
    X_cudf = X_cudf.fillna(X_cudf.median())
    y_cudf = y_cudf.fillna(y_cudf.median())

    # Convert to cupy
    X_cp = X_cudf.to_cupy()
    y_cp = y_cudf.to_cupy()

    # Rank transform
    X_rank = cp.zeros_like(X_cp)
    for i in range(X_cp.shape[1]):
        X_rank[:, i] = cupy_rankdata(X_cp[:, i])

    y_rank = cupy_rankdata(y_cp)

    # Center ranks
    X_rank -= cp.mean(X_rank, axis=0)
    y_rank -= cp.mean(y_rank)

    # Spearman correlation
    numerator = cp.sum(X_rank * y_rank[:, None], axis=0)
    denominator = cp.sqrt(cp.sum(X_rank ** 2, axis=0)) * cp.sqrt(cp.sum(y_rank ** 2))

    spearman_corr = numerator / (denominator + 1e-10)

    # MI proxy = abs(Spearman correlation)
    mi_df = pd.DataFrame({
        'feature': feature_cols,
        'mi_score': cp.abs(spearman_corr).get()
    }).sort_values(by='mi_score', ascending=False)

    del X_cudf, y_cudf, X_cp, y_cp, X_rank, y_rank
    gc.collect()

    return mi_df

# Step 3: SHAP Feature Selection (Interaction-aware)
def get_shap_importance(train_df, feature_cols, n_samples=10000):
    sample_df = train_df.sample(n=min(n_samples, len(train_df)), random_state=42)
    
    X_sample = sample_df[feature_cols].astype(np.float32)
    y_sample = sample_df['y'].astype(np.int8)

    params = {
        'objective': 'binary',
        'boosting_type': 'gbdt',
        'learning_rate': 0.05,
        'num_leaves': 31,
        'max_depth': -1,
        'feature_fraction': 0.8,
        'bagging_fraction': 0.8,
        'bagging_freq': 5,
        'min_data_in_leaf': 100,
        'verbosity': -1,
        'device': 'gpu',
        'gpu_platform_id': 0,
        'gpu_device_id': 0,
        'seed': 42
    }

    dtrain = lgb.Dataset(X_sample, label=y_sample)
    model = lgb.train(params, dtrain, num_boost_round=100)

    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(X_sample)[1]  # Binary classification -> index 1 is for positive class

    shap_importance = np.abs(shap_values).mean(axis=0)
    shap_df = pd.DataFrame({
        'feature': feature_cols,
        'shap_value': shap_importance
    }).sort_values(by='shap_value', ascending=False)

    del X_sample, y_sample, dtrain, model, explainer, shap_values
    gc.collect()
    
    return shap_df

# Step 4: Run the pipeline
def run_feature_selection_pipeline(train_df, top_k=200):
    print("Extracting feature columns...")
    feature_cols = [col for col in train_df.columns if col not in ['id1', 'id2', 'id3', 'id4', 'id5', 'y', 'pred'] and not col.endswith('_dt')]

    print("Step 1: LightGBM GPU Importance...")
    lgb_top_features, lgb_df = get_lgb_gpu_importance(train_df, feature_cols)

    print("Step 2: Mutual Information...")
    # Step 2: Mutual Info Proxy GPU
    mi_df = gpu_mutual_info_proxy(train_df, feature_cols)


    print("Step 3: SHAP Importance...")
    shap_df = get_shap_importance(train_df, feature_cols)

    print("Step 4: Aggregate & Select Top Features...")

    # Normalize scores for fair aggregation
    lgb_df['norm_imp'] = lgb_df['importance'] / (lgb_df['importance'].sum() + 1e-10)
    mi_df['norm_mi'] = mi_df['mi_score'] / (mi_df['mi_score'].sum() + 1e-10)
    shap_df['norm_shap'] = shap_df['shap_value'] / (shap_df['shap_value'].sum() + 1e-10)

    # Merge scores
    agg = lgb_df[['feature', 'norm_imp']].merge(
        mi_df[['feature', 'norm_mi']], on='feature', how='outer'
    ).merge(
        shap_df[['feature', 'norm_shap']], on='feature', how='outer'
    ).fillna(0)

    # Weighted ensemble of scores
    agg['total_score'] = 0.4 * agg['norm_imp'] + 0.3 * agg['norm_mi'] + 0.3 * agg['norm_shap']
    agg = agg.sort_values('total_score', ascending=False)

    selected_features = agg.head(top_k)['feature'].tolist()

    print(f"Selected top {top_k} features.")
    
    return selected_features, agg


selected_features, feature_score_df = run_feature_selection_pipeline(train_df, top_k=200)


In [ ]:
selected_features

In [ ]:
gc.collect()


In [ ]:
id_cols = ['id1', 'id2', 'id3', 'id4', 'id5']

# For Train
train_filtered = train_df[selected_features + id_cols + ['y']].copy()

# For Validation
val_filtered = val_df[selected_features + id_cols + ['y']].copy()

# For Test (no 'y')
test_filtered = test_df[selected_features + id_cols].copy()


In [ ]:
train_filtered.shape

In [ ]:
import pandas as pd
import numpy as np
from numba import jit
import warnings
warnings.filterwarnings('ignore')

def create_fast_lag_features(train_df: pd.DataFrame, val_df: pd.DataFrame, test_df: pd.DataFrame,
                           target_col: str = 'y') -> tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """
    Create essential lag features optimized for speed and large imbalanced datasets.
    Focus on features that provide maximum signal with minimum computation.
    
    Parameters:
    - train_df, val_df, test_df: DataFrames with time series data
    - target_col: Name of target column (to avoid data leakage)
    
    Returns:
    - Tuple of enhanced DataFrames with essential lag features
    """
    print("=== Creating Fast Essential Lag Features ===")
    
    # Create copies
    train_df = train_df.copy()
    val_df = val_df.copy()
    test_df = test_df.copy()
    
    # Ensure datetime
    for df in [train_df, val_df, test_df]:
        df['id4'] = pd.to_datetime(df['id4'], errors='coerce')
    
    # Define ONLY the most important features for lag creation
    # Focus on behavioral signals that are most predictive
    essential_features = [
        'f132',           # Core transaction amount
        'best_day_ctr',   # Key performance metric
        'overall_ctr',    # Overall engagement
    ]
    
    # Add GPU features if they exist (these are usually high-value)
    gpu_features = ['customer_avg_trans', 'customer_trans_count']
    for feature in gpu_features:
        if feature in train_df.columns:
            essential_features.append(feature)
    
    # Filter to only existing features
    # This line should be added to filter out the target column:
    available_features = [f for f in essential_features 
                     if f in train_df.columns and f != target_col]
    print(f"Creating lag features for {len(available_features)} high-value features: {available_features}")
    
    def create_minimal_lags(df):
        """Create only the most essential lag features for speed"""
        df_enhanced = df.copy()
        df_enhanced = df_enhanced.sort_values(['id2', 'id4'])
        
        print("   Creating essential lag features...")
        
        # 1. ONLY most critical lags (1, 7 days)
        critical_lags = [1, 7]  # Reduced from [1, 2, 3, 7, 14]
        
        for feature in available_features:
            # Basic lags - only most important ones
            for lag in critical_lags:
                df_enhanced[f'{feature}_lag_{lag}'] = df_enhanced.groupby('id2')[feature].shift(lag)
            
            # Only short-term rolling mean (most predictive)
            df_enhanced[f'{feature}_roll3'] = (
                df_enhanced.groupby('id2')[feature]
                .rolling(window=3, min_periods=1)
                .mean()
                .reset_index(level=0, drop=True)
            )
            
            # Recent change (most important momentum signal)
            df_enhanced[f'{feature}_diff1'] = df_enhanced.groupby('id2')[feature].diff(1)
            
            # Current vs recent average (key signal for imbalanced data)
            df_enhanced[f'{feature}_vs_avg'] = (
                df_enhanced[feature] - df_enhanced[f'{feature}_roll3']
            )
        
        print("   Creating cross-feature ratios...")
        # 2. ONLY most important cross-feature interactions
        if 'best_day_ctr' in df_enhanced.columns and 'overall_ctr' in df_enhanced.columns:
            df_enhanced['ctr_ratio_lag1'] = (
                df_enhanced['best_day_ctr_lag_1'] / (df_enhanced['overall_ctr_lag_1'] + 1e-6)
            )
        
        if 'f132' in df_enhanced.columns:
            # Transaction amount momentum
            df_enhanced['f132_momentum'] = (
                df_enhanced['f132'] / (df_enhanced['f132_roll3'] + 1e-6)
            )
        
        return df_enhanced
    
    def create_imbalanced_features(df):
        """Create features specifically useful for imbalanced datasets"""
        df_enhanced = df.copy()
        
        print("   Creating imbalanced-specific features...")
        
        # Customer behavior consistency (helps identify anomalies in imbalanced data)
        for feature in available_features:
            if f'{feature}_roll3' in df_enhanced.columns:
                # Coefficient of variation (stability measure)
                rolling_std = (
                    df_enhanced.groupby('id2')[feature]
                    .rolling(window=3, min_periods=2)
                    .std()
                    .reset_index(level=0, drop=True)
                )
                df_enhanced[f'{feature}_stability'] = (
                    rolling_std / (df_enhanced[f'{feature}_roll3'] + 1e-6)
                )
        
        # Recent activity level (important for rare events)
        if 'f132' in available_features:
            df_enhanced['recent_activity'] = (
                df_enhanced.groupby('id2')['f132']
                .rolling(window=3, min_periods=1)
                .count()
                .reset_index(level=0, drop=True)
            )
        
        return df_enhanced
    
    def fast_fill_missing(df):
        """Fast missing value filling optimized for large datasets"""
        # Use median filling (faster than forward fill for large data)
        lag_cols = [col for col in df.columns if any(x in col for x in ['_lag_', '_roll', '_diff', '_vs_', '_stability', '_momentum'])]
        
        for col in lag_cols:
            if col.endswith('_diff1'):
                df[col] = df[col].fillna(0)  # No change
            elif 'stability' in col or 'momentum' in col:
                df[col] = df[col].fillna(1)  # Neutral values
            else:
                df[col] = df[col].fillna(df[col].median())
        
        return df
    
    # Apply to all datasets
    print("\nProcessing training data...")
    train_enhanced = create_minimal_lags(train_df)
    train_enhanced = create_imbalanced_features(train_enhanced)
    train_enhanced = fast_fill_missing(train_enhanced)
    
    print("Processing validation data...")
    val_enhanced = create_minimal_lags(val_df)
    val_enhanced = create_imbalanced_features(val_enhanced)
    val_enhanced = fast_fill_missing(val_enhanced)
    
    print("Processing test data...")
    test_enhanced = create_minimal_lags(test_df)
    test_enhanced = create_imbalanced_features(test_enhanced)
    test_enhanced = fast_fill_missing(test_enhanced)
    
    # Summary
    original_cols = len(train_df.columns)
    new_cols = len(train_enhanced.columns) - original_cols
    
    print(f"\n=== Fast Lag Feature Creation Complete ===")
    print(f"New features created: {new_cols} (optimized for speed)")
    print(f"Training shape: {train_enhanced.shape}")
    print(f"Features per base feature: ~{new_cols // len(available_features)} (vs ~50+ in full version)")
    
    return train_enhanced, val_enhanced, test_enhanced


def create_ultra_fast_features(train_df: pd.DataFrame, val_df: pd.DataFrame, test_df: pd.DataFrame) -> tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """
    Ultra-fast version with only the most critical features for very large datasets.
    Creates ~3-5 features per base feature instead of 50+.
    """
    print("=== Creating Ultra-Fast Critical Features ===")
    
    train_df = train_df.copy()
    val_df = val_df.copy()
    test_df = test_df.copy()
    
    # Ensure datetime
    for df in [train_df, val_df, test_df]:
        df['id4'] = pd.to_datetime(df['id4'], errors='coerce')
    
    # ONLY the most critical features
    critical_features = ['f132', 'best_day_ctr']
    available = [f for f in critical_features if f in train_df.columns]
    
    def create_critical_only(df):
        df_enhanced = df.copy()
        df_enhanced = df_enhanced.sort_values(['id2', 'id4'])
        
        for feature in available:
            # Only 1-day lag (most important)
            df_enhanced[f'{feature}_lag1'] = df_enhanced.groupby('id2')[feature].shift(1)
            
            # Only 3-day rolling average
            df_enhanced[f'{feature}_avg3'] = (
                df_enhanced.groupby('id2')[feature]
                .rolling(window=3, min_periods=1)
                .mean()
                .reset_index(level=0, drop=True)
            )
            
            # Current vs recent (key momentum signal)
            df_enhanced[f'{feature}_momentum'] = (
                df_enhanced[feature] / (df_enhanced[f'{feature}_avg3'] + 1e-6)
            )
        
        # Fill missing with median (fastest)
        new_cols = [col for col in df_enhanced.columns if col not in df.columns]
        for col in new_cols:
            df_enhanced[col] = df_enhanced[col].fillna(df_enhanced[col].median())
        
        return df_enhanced
    
    # Process all datasets
    train_enhanced = create_critical_only(train_df)
    val_enhanced = create_critical_only(val_df)
    test_enhanced = create_critical_only(test_df)
    
    new_features = len(train_enhanced.columns) - len(train_df.columns)
    print(f"Created {new_features} ultra-fast features")
    print(f"Training shape: {train_enhanced.shape}")
    
    return train_enhanced, val_enhanced, test_enhanced

train_filtered, val_filtered, test_filtered = create_fast_lag_features(train_filtered, val_filtered, test_filtered)

In [ ]:
import gc

# Delete old datasets explicitly
del train_df
del val_df
del test_df

# Run garbage collection to free memory
gc.collect()

# Optional: Clear CuPy GPU memory if you're using it
import cupy as cp
cp._default_memory_pool.free_all_blocks()

print("Old datasets deleted and memory freed.")


In [ ]:

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix, precision_score, recall_score, f1_score
import lightgbm as lgb
import time


def train_model(train_df: pd.DataFrame, val_df: pd.DataFrame) -> lgb.Booster:
    """Train LightGBM model for ranking with target 'y', calculate ranking metrics,
    and dynamically select the optimal threshold based on F1-score."""
    
    # Get feature columns (exclude target column 'y', ID columns except 'id2', and date columns)
    feature_cols = [col for col in train_df.columns if col not in ['id1','id2','id3', 'id4', 'id5', 'id6', 'id7', 'id8','id9', 'id10', 'id11', 'id12', 'id13','y', 'pred'] and not col.endswith('_dt')]
    
    # Prepare data and get categorical columns
    train_df, val_df = prepare_data_for_lgb(train_df, val_df)
    
    print(f"Training data: {len(train_df)} rows, {len(feature_cols)} features")
    print(f"Validation data: {len(val_df)} rows")
    # print(f"Categorical columns passed to LightGBM: {categorical_cols}")
    
    # Check target distribution
    train_target_dist = train_df['y'].value_counts().sort_index()
    val_target_dist = val_df['y'].value_counts().sort_index()
    print(f"Train target distribution: {train_target_dist.to_dict()}")
    print(f"Val target distribution: {val_target_dist.to_dict()}")
    
    # Calculate scale_pos_weight for imbalanced data
    pos_samples = len(train_df[train_df['y'] == 1])
    neg_samples = len(train_df[train_df['y'] == 0])
    scale_pos_weight = neg_samples / pos_samples if pos_samples > 0 else 1.0
    
    # Optimized LightGBM parameters for ranking
    params = {
    'objective': 'lambdarank',
    'metric': 'map',
    'eval_at': [7],
    'lambdarank_truncation_level': 7,
    'lambdarank_norm': True,

    'boosting_type': 'gbdt',
    'num_leaves': 63,                # Restore leaves for more splits
    'learning_rate': 0.03,           # Increase LR for faster learning
    'max_depth': -1,                 # Allow deeper trees

    'feature_fraction': 0.8,         # Slightly more features per split
    'bagging_fraction': 0.8,
    'bagging_freq': 5,

    'lambda_l1': 0.1,                # Relax L1
    'lambda_l2': 0.1,                # Relax L2
    'min_data_in_leaf': 50,          # Reduce from 200 to 50
    'min_gain_to_split': 0.0,        # Allow any positive gain
    'max_bin': 255,

    'force_col_wise': True,
    'num_threads': -1,
    'verbosity': 1,
    'seed': 42,
    }

    print(f"Scale pos weight: {scale_pos_weight:.4f}")
    
    # Sort dataframes by customer for proper grouping
    train_df = train_df.sort_values('id2').reset_index(drop=True)
    val_df = val_df.sort_values('id2').reset_index(drop=True)
    
    # Create group information for ranking
    train_groups = train_df.groupby('id2').size().values
    val_groups = val_df.groupby('id2').size().values
    
    print(f"Train groups: {len(train_groups)} customers, avg {train_groups.mean():.1f} offers per customer")
    print(f"Val groups: {len(val_groups)} customers, avg {val_groups.mean():.1f} offers per customer")
    
    # Create datasets
    print("Creating LightGBM datasets...")
    dtrain = lgb.Dataset(train_df[feature_cols], label=train_df['y'], group=train_groups, free_raw_data=False)
    dval = lgb.Dataset(val_df[feature_cols], label=val_df['y'], group=val_groups, reference=dtrain, free_raw_data=False)
    
    dtrain.construct()
    dval.construct()
    
    print(f"Training dataset created: {dtrain.num_data()} samples")
    print(f"Validation dataset created: {dval.num_data()} samples")
    
    # Train model
    print("Starting LightGBM training...")
    start_time = time.time()
    model = lgb.train(
        params,
        dtrain,
        num_boost_round=3000,
        valid_sets=[dtrain, dval],
        valid_names=['train', 'val'],
        callbacks=[
            lgb.early_stopping(stopping_rounds=200, verbose=True),
            lgb.log_evaluation(period=50)
        ]
    )
    
    training_time = time.time() - start_time
    print(f"Training completed in {training_time:.2f} seconds")
    print(f"Best iteration: {model.best_iteration}")
    print(f"Best score: {model.best_score}")
    
    # Calculate ranking scores
    print("\n" + "="*50)
    print("RANKING EVALUATION (Using Ranking Scores)")
    print("="*50)
    gc.collect()
    # Predictions on training set (ranking scores)
    train_pred = model.predict(train_df[feature_cols], num_iteration=model.best_iteration)
    train_auc = roc_auc_score(train_df['y'], train_pred)
    gc.collect()
    # Predictions on validation set (ranking scores)
    val_pred = model.predict(val_df[feature_cols], num_iteration=model.best_iteration)
    val_auc = roc_auc_score(val_df['y'], val_pred)
    gc.collect()
    print(f"Training AUC: {train_auc:.6f}")
    print(f"Validation AUC: {val_auc:.6f}")
    print(f"AUC Difference (Train - Val): {train_auc - val_auc:.6f}")
    
    # Dynamic threshold selection based on F1-score
    print("\n" + "="*50)
    print("DYNAMIC THRESHOLD SELECTION (Maximizing F1-Score)")
    print("="*50)
    gc.collect()
    thresholds = np.arange(0.1, 0.91, 0.01)
    best_threshold = 0.5
    best_f1 = 0.0
    best_precision = 0.0
    best_recall = 0.0
    
    for threshold in thresholds:
        val_pred_binary = (val_pred > threshold).astype(int)
        f1 = f1_score(val_df['y'], val_pred_binary)
        if f1 > best_f1:
            best_f1 = f1
            best_threshold = threshold
            best_precision = precision_score(val_df['y'], val_pred_binary)
            best_recall = recall_score(val_df['y'], val_pred_binary)
    
    print(f"Optimal threshold: {best_threshold:.2f}")
    print(f"Best F1-Score: {best_f1:.6f}")
    print(f"Precision at optimal threshold: {best_precision:.6f}")
    print(f"Recall at optimal threshold: {best_recall:.6f}")
    gc.collect()
    # Generate binary predictions using the optimal threshold
    val_pred_binary = (val_pred > best_threshold).astype(int)
    
    print("\n" + "="*50)
    print(f"CLASSIFICATION REPORT (Validation Set, Threshold={best_threshold:.2f})")
    print("="*50)
    print(classification_report(val_df['y'], val_pred_binary))
    
    print("\nConfusion Matrix (Validation Set):")
    cm = confusion_matrix(val_df['y'], val_pred_binary)
    print(cm)
    gc.collect()
    # Store metrics in model object for later access
    model.train_auc = train_auc
    model.val_auc = val_auc
    model.val_precision = best_precision
    model.val_recall = best_recall
    model.val_f1 = best_f1
    model.optimal_threshold = best_threshold
    
    # Feature importance
    importance = model.feature_importance(importance_type='gain')
    feature_importance = dict(zip(feature_cols, importance))
    
    # Print top 10 most important features
    print("\n" + "="*50)
    print("TOP 10 MOST IMPORTANT FEATURES")
    print("="*50)
    for i, (feature, importance) in enumerate(sorted(feature_importance.items(), key=lambda x: x[1], reverse=True)[:10], 1):
        print(f"{i:2d}. {feature}: {importance:.2f}")
    
    return model

In [ ]:
print("=" * 50)
print("STARTING MODEL TRAINING")
print("=" * 50)

results = train_model(train_filtered, val_filtered)

print("=" * 50)
print("MODEL TRAINING COMPLETED")
print("=" * 50)



In [ ]:
def select_top_features(train_df, model, k=50):
    """
    Select top k features based on LightGBM feature importance.
    
    Parameters:
    - train_df: Training DataFrame
    - model: Trained LightGBM model
    - k: Number of top features to select
    
    Returns:
    - List of top k feature names
    """
    feature_cols = [col for col in train_df.columns if col not in ['y', 'id1', 'id3', 'id4', 'id5'] and not col.endswith('_dt')]
    importance = model.feature_importance(importance_type='gain')
    feature_importance = dict(zip(feature_cols, importance))
    sorted_features = sorted(feature_importance.items(), key=lambda x: x[1], reverse=True)
    top_features = [feature for feature, _ in sorted_features[:k]]
    print(f"Top {k} features: {top_features}")
    return top_features

# Select top 10 features
features = select_top_features(train_df, results, k=150)

In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb

def calculate_map7(df: pd.DataFrame, model: lgb.Booster, use_threshold: bool = False) -> float:
    """
    Calculate MAP@7 metric for the validation DataFrame using LightGBM probability predictions
    
    Parameters:
    - df: DataFrame containing validation data with 'id2' (customer ID) and 'y' columns
    - model: Trained LightGBM model
    - use_threshold: If True, use model's optimal threshold for ranking; if False, use raw probabilities
    
    Returns:
    - Float representing the MAP@7 score
    """
    print("Calculating MAP@7...")
    print(f"Data shape: {df.shape}")
    
    # Check if id2 exists
    if 'id2' not in df.columns:
        raise KeyError("Column 'id2' not found in DataFrame. Ensure 'id2' is not dropped during preprocessing.")
    
    # Automatically select feature columns (same as train_model)
    exclude_cols = ['id1','id2', 'id3', 'id4', 'id5', 'id6', 'id7', 'id8', 
                    'id9', 'id10', 'id11', 'id12', 'id13', 'pred', 'y', 'y_pred']
    exclude_cols.extend([col for col in df.columns if col.endswith('_dt')])
    feature_cols = [col for col in df.columns if col not in exclude_cols]
    
    print(f"Using {len(feature_cols)} features for prediction")
    
    # Generate probability predictions
    df = df.copy()
    df['y_pred'] = model.predict(df[feature_cols], num_iteration=model.best_iteration)
    
    # If use_threshold is True, convert probabilities to binary scores using optimal threshold
    if use_threshold and hasattr(model, 'optimal_threshold'):
        print(f"Using optimal threshold for ranking: {model.optimal_threshold:.4f}")
        df['y_pred'] = (df['y_pred'] > model.optimal_threshold).astype(int)
    
    # Verify prediction range
    print(f"Prediction range: min={df['y_pred'].min():.6f}, max={df['y_pred'].max():.6f}, mean={df['y_pred'].mean():.6f}")
    
    customers = df['id2'].unique()
    print(f"Number of customers: {len(customers)}")
    
    map_scores = []
    customers_with_clicks = 0
    total_clicks = 0
    
    for i, customer_id in enumerate(customers):
        customer_df = df[df['id2'] == customer_id].copy()
        
        # Sort by prediction scores in descending order
        customer_df = customer_df.sort_values('y_pred', ascending=False)
        
        # Get top 7 predictions
        top7 = customer_df.head(7)
        
        # Calculate Average Precision (AP)
        customer_clicks = top7['y'].sum()
        total_clicks += customer_clicks
        
        if customer_clicks > 0:
            customers_with_clicks += 1
            precisions = []
            hits = 0
            for j, (_, row) in enumerate(top7.iterrows()):
                if row['y'] == 1:
                    hits += 1
                    precisions.append(hits / (j + 1))
            
            if precisions:
                map_scores.append(np.mean(precisions))
        
        # Progress logging
        if (i + 1) % 1000 == 0:
            print(f"Processed {i+1}/{len(customers)} customers...")
    
    final_map7 = np.mean(map_scores) if map_scores else 0.0
    
    print("MAP@7 calculation complete:")
    print(f"  - Customers with clicks in top-7: {customers_with_clicks}/{len(customers)}")
    print(f"  - Total clicks in top-7: {total_clicks}")
    print(f"  - MAP@7 score: {final_map7:.6f}")

    return final_map7


In [ ]:
# map7_score = calculate_map7(val_df, model,use_threshold=True)
# print(f"MAP@7 Score (Raw Probabilities): {map7_score:.6f}")
map7_score = calculate_map7(val_filtered,results, use_threshold=False)

In [ ]:
# template = pd.read_csv('/kaggle/input/amex-data/template.csv')
# template.head()

In [ ]:
# %% [code]
def prepare_test_data_for_lgb(test_df: pd.DataFrame, train_df: pd.DataFrame) -> pd.DataFrame:
    """
    Prepare test data for LightGBM by converting object columns to numeric
    Uses the same encoding as training data to ensure consistency
    """
    test_df_copy = test_df.copy()
    train_df_copy = train_df.copy()
    
    # Get feature columns (exclude target 'y' and ID columns)
    feature_cols = [col for col in test_df_copy.columns if col not in ['y','id1', 'id2', 'id3', 'id4', 'id5']]
    
    # Find object columns
    object_cols = []
    for col in feature_cols:
        if test_df_copy[col].dtype == 'object':
            object_cols.append(col)
    
    print(f"Found {len(object_cols)} object columns to convert in test data:")
    print(object_cols)
    
    # Convert object columns to numeric
    for col in object_cols:
        print(f"Converting column: {col}")
        
        # Combine train and test data for consistent encoding
        combined_data = pd.concat([train_df_copy[col] if col in train_df_copy.columns else pd.Series(), 
                                   test_df_copy[col]], axis=0)
        
        # Try to convert to numeric first
        try:
            combined_numeric = pd.to_numeric(combined_data, errors='coerce')
            if combined_numeric.notna().sum() > len(combined_data) * 0.8:
                test_df_copy[col] = pd.to_numeric(test_df_copy[col], errors='coerce')
                print(f"  -> Converted to numeric")
                continue
        except:
            pass
        
        # If not numeric, use label encoding
        le = LabelEncoder()
        combined_data_filled = combined_data.fillna('missing').astype(str)
        le.fit(combined_data_filled)
        
        test_filled = test_df_copy[col].fillna('missing').astype(str)
        test_filled_encoded = []
        for val in test_filled:
            if val in le.classes_:
                test_filled_encoded.append(le.transform([val])[0])
            else:
                test_filled_encoded.append(-1)  # Use -1 for unseen labels
        
        test_df_copy[col] = test_filled_encoded
        print(f"  -> Label encoded ({len(le.classes_)} unique values)")
    
    # Convert any remaining object columns to numeric
    for col in feature_cols:
        if test_df_copy[col].dtype == 'object':
            test_df_copy[col] = pd.to_numeric(test_df_copy[col], errors='coerce').fillna(0)
    
    print("\nFinal test data types:")
    feature_dtypes = test_df_copy[feature_cols].dtypes
    object_remaining = feature_dtypes[feature_dtypes == 'object']
    if len(object_remaining) > 0:
        print(f"WARNING: Still have object columns: {object_remaining.index.tolist()}")
    else:
        print("✓ All feature columns are numeric")
    
    return test_df_copy

# %% [code]
print("=" * 50)
print("PREPARING TEST DATA")
print("=" * 50)
test_df_prepared = prepare_test_data_for_lgb(test_filtered, train_filtered)

# %% [code]
print("=" * 50)
print("DEBUGGING: ALIGNING FEATURES TO MODEL")
print("=" * 50)

# Get model features directly from the trained model
model_features = results.feature_name()
print(f"Model expects {len(model_features)} features.")

# Add missing features in test data
for col in model_features:
    if col not in test_df_prepared.columns:
        print(f"Adding missing feature '{col}' to test data with value 0")
        test_df_prepared[col] = 0

# Drop extra features not used in model
extra_features = [col for col in test_df_prepared.columns if col not in model_features + ['id1', 'id2', 'id3', 'id4', 'id5']]
if extra_features:
    print(f"Dropping extra features from test data: {extra_features}")
    test_df_prepared = test_df_prepared.drop(columns=extra_features)

# Reorder test columns to match model order
test_df_prepared = test_df_prepared[['id1', 'id2', 'id3', 'id4', 'id5'] + model_features]

print("Final test data shape:", test_df_prepared.shape)

# %% [code]
from scipy.special import expit  # For sigmoid

def generate_test_predictions(test_df: pd.DataFrame, model: lgb.Booster) -> pd.DataFrame:
    print("=" * 50)
    print("GENERATING TEST PREDICTIONS")
    print("=" * 50)
    
    feature_cols = model.feature_name()
    
    print(f"Test data shape: {test_df.shape}")
    print(f"Using {len(feature_cols)} features for prediction")
    
    raw_preds = model.predict(test_df[feature_cols], num_iteration=model.best_iteration)
    
    # Apply sigmoid to convert raw scores to probabilities
    sigmoid_preds = expit(raw_preds)
    
    # Clip for numerical stability
    test_predictions = np.clip(sigmoid_preds, 0.0, 1.0)
    
    print(f"Prediction range after sigmoid: min={test_predictions.min():.6f}, max={test_predictions.max():.6f}, mean={test_predictions.mean():.6f}")
    
    submission_df = pd.DataFrame({
        'id1': test_df['id1'],
        'id2': test_df['id2'],
        'id3': test_df['id3'],
        'id5': test_df['id5'],
        'pred': test_predictions
    })
    
    return submission_df


# %% [code]
submission_df = generate_test_predictions(test_df_prepared, results)

# %% [code]
print("=" * 50)
print("VERIFYING SUBMISSION FORMAT")
print("=" * 50)

template_df = pd.read_csv('/kaggle/input/amex-data/template.csv')

if submission_df.shape[0] == template_df.shape[0]:
    print("✓ Row count matches template")
else:
    print(f"WARNING: Row count mismatch! Template: {template_df.shape[0]}, Submission: {submission_df.shape[0]}")

if list(submission_df.columns) == list(template_df.columns):
    print("✓ Column names and order match template")
else:
    print("WARNING: Column mismatch!")
    print(f"Template columns: {template_df.columns.tolist()}")
    print(f"Submission columns: {submission_df.columns.tolist()}")

print("\nSample predictions:")
print(submission_df.head(10))

print(f"\nPrediction stats: Min={submission_df['pred'].min():.6f}, Max={submission_df['pred'].max():.6f}, Mean={submission_df['pred'].mean():.6f}, Std={submission_df['pred'].std():.6f}")

# %% [code]
submission_path = '/kaggle/working/submission.csv'
submission_df.to_csv(submission_path, index=False)
print(f"\n✓ Submission saved to: {submission_path}")

print("\nFinal submission info:")
print(f"Total rows: {len(submission_df):,}")
print(f"Unique id2: {submission_df['id2'].nunique():,}")
print(f"Unique id3: {submission_df['id3'].nunique():,}")

if submission_df.isnull().any().any():
    print("\nWARNING: Missing values detected in submission!")
    print(submission_df.isnull().sum())
else:
    print("\n✓ No missing values in submission")

print("\n" + "=" * 50)
print("SUBMISSION PREPARATION COMPLETE!")
print("=" * 50)


In [ ]:
gc.collect()